In [ ]:
!nvidia-smi
!pip -q install soundfile librosa tqdm scipy thop fvcore torchcodec

Wed Jan 14 12:29:36 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import os, sys

# EfficientAT (for MobileNetV3 + DyMN + AugmentMelSTFT)
if not os.path.exists("EfficientAT"):
    !git clone --depth 1 https://github.com/fschmid56/EfficientAT.git

# REQUIRED: remove the following code in EfficientAT/helpers/utils.py (line 35-50)
# import csv

# # Load label
# with open('metadata/class_labels_indices.csv', 'r') as f:
#     reader = csv.reader(f, delimiter=',')
#     lines = list(reader)

# labels = []
# ids = []    # Each label has a unique id such as "/m/068hy"
# for i1 in range(1, len(lines)):
#     id = lines[i1][1]
#     label = lines[i1][2]
#     ids.append(id)
#     labels.append(label)

# classes_num = len(labels)

sys.path.append("/content/EfficientAT")

# Optional: silence wandb if it tries to init anywhere
os.environ["WANDB_SILENT"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
import re
import json
import math
import time
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torchaudio

import soundfile as sf
from scipy import signal

# EfficientAT imports
from models.preprocess import AugmentMelSTFT
from models.mn.model import get_model as get_mobilenet
from models.dymn.model import get_model as get_dymn
from helpers.utils import NAME_TO_WIDTH

# FLOPs/MACs helpers
from fvcore.nn import FlopCountAnalysis

def set_seed(seed: int = 1337):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(1337)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE =", DEVICE)

DEVICE = cuda


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# ESC-50 (preprocessed to 32k) root folder:
#  <ESC50_32K_ROOT>/
#     audio_32k/
#     meta/esc50.csv
ESC50_32K_ROOT = "/content/drive/MyDrive/ESC-50/esc50-32k"

# If you prefer using the uploaded csv you attached (optional):
ESC50_CSV_FALLBACK = "/mnt/data/esc50.csv"  # works only if present in your runtime

# Stage 2: BC-ResNet binary danger/safe (tau=1,3,8)
# train_esc50_binary.py must be in the same working dir (/content)
BCRESNET_CKPTS = {
    "bcresnet_tau1": "/content/bcresnet1.pt",
    "bcresnet_tau3": "/content/bcresnet3.pt",
    "bcresnet_tau8": "/content/bcresnet8.pt"
}

# Stage 2: MobileNetV3 (0.4) binary danger/safe (your fine-tuned checkpoint)
STAGE2_MN_MODEL_NAME = "mn04_as"  # width=0.4 convention in EfficientAT
STAGE2_MN_CKPT = "/content/mn04_as.pt"
STAGE2_MN_NUM_CLASSES = 2

# Stage 3: DyMN-M (your fine-tuned checkpoint, 50-way ESC-50)
STAGE3_DYMN_MODEL_NAME = "dymn10_as"  # change if you trained a different name
STAGE3_DYMN_CKPT = "/content/dymn10_as.pt"
STAGE3_NUM_CLASSES = 50

# Output directory
OUT_DIR = "/content/echo_guard_runs"
os.makedirs(OUT_DIR, exist_ok=True)

print("OUT_DIR:", OUT_DIR)


OUT_DIR: /content/echo_guard_runs


In [ ]:
def resolve_esc50_csv(root: str) -> str:
    p1 = os.path.join(root, "meta", "esc50.csv")
    if os.path.exists(p1):
        return p1
    if os.path.exists(ESC50_CSV_FALLBACK):
        return ESC50_CSV_FALLBACK
    raise FileNotFoundError("Cannot find esc50.csv. Put it under <root>/meta/esc50.csv or set ESC50_CSV_FALLBACK.")

def resolve_audio_path(root: str, filename: str) -> str:
    # your dataset uses audio_32k/
    p = os.path.join(root, "audio_32k", filename)
    if not os.path.exists(p):
        raise FileNotFoundError(f"Missing audio file: {p}")
    return p

esc_csv = resolve_esc50_csv(ESC50_32K_ROOT)
df = pd.read_csv(esc_csv)

required_cols = {"filename", "fold", "target", "category"}
assert required_cols.issubset(df.columns), f"esc50.csv missing cols: {required_cols - set(df.columns)}"

print("ESC50 rows:", len(df))
print(df.head())
print("Unique categories:", df["category"].nunique())


ESC50 rows: 2000
            filename  fold  target        category  esc10  src_file take
0   1-100032-A-0.wav     1       0             dog   True    100032    A
1  1-100038-A-14.wav     1      14  chirping_birds  False    100038    A
2  1-100210-A-36.wav     1      36  vacuum_cleaner  False    100210    A
3  1-100210-B-36.wav     1      36  vacuum_cleaner  False    100210    B
4  1-101296-A-19.wav     1      19    thunderstorm  False    101296    A
Unique categories: 50


In [ ]:
# Your FIXED emergency/danger labels (ESC-50 categories)
EMERGENCY_CATS = {
    "siren",
    "car_horn",
    "glass_breaking",
    "thunderstorm",
    "crying_baby",
    "dog",
    "door_wood_knock",
    "clock_alarm",
}

# Build target<->category mapping from esc50.csv
target_to_cat = dict(df[["target", "category"]].drop_duplicates().values.tolist())
cat_to_target = {v: k for k, v in target_to_cat.items()}

# sanity: ensure all emergency classes exist
missing = sorted([c for c in EMERGENCY_CATS if c not in cat_to_target])
assert len(missing) == 0, f"These emergency categories not found in esc50.csv: {missing}"

df["is_emergency"] = df["category"].isin(EMERGENCY_CATS).astype(int)

print("Emergency clips:", int(df["is_emergency"].sum()), "/", len(df))
print("Emergency targets:", {c: cat_to_target[c] for c in sorted(EMERGENCY_CATS)})


Emergency clips: 320 / 2000
Emergency targets: {'car_horn': 43, 'clock_alarm': 37, 'crying_baby': 20, 'dog': 0, 'door_wood_knock': 30, 'glass_breaking': 39, 'siren': 42, 'thunderstorm': 19}


In [ ]:
SR = 32000
CLIP_S = 5.0
CLIP_T = int(SR * CLIP_S)

def load_audio_32k_5s(path: str, sr: int = SR, T: int = CLIP_T) -> np.ndarray:
    wav, file_sr = sf.read(path, dtype="float32", always_2d=False)
    if wav.ndim > 1:
        wav = wav.mean(axis=-1)  # mono

    if file_sr != sr:
        wav_t = torch.from_numpy(wav)
        wav = torchaudio.functional.resample(wav_t, file_sr, sr).numpy().astype(np.float32)

    if len(wav) < T:
        wav = np.pad(wav, (0, T - len(wav)))
    elif len(wav) > T:
        # center crop for determinism
        start = (len(wav) - T) // 2
        wav = wav[start:start+T]

    wav = np.clip(wav, -1.0, 1.0).astype(np.float32)
    return wav


In [ ]:
def strip_module_prefix(state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    return {k.replace("module.", ""): v for k, v in state.items()}

def extract_state_dict(ckpt: Any) -> Dict[str, torch.Tensor]:
    # Supports: raw state_dict, or dict containing common keys
    if isinstance(ckpt, dict):
        if all(isinstance(v, torch.Tensor) for v in ckpt.values()):
            return strip_module_prefix(ckpt)
        for key in ["state_dict", "model", "model_state_dict", "net", "weights"]:
            if key in ckpt and isinstance(ckpt[key], dict):
                return strip_module_prefix(ckpt[key])
        # last resort: tensor-like entries
        tensor_like = {k: v for k, v in ckpt.items() if isinstance(v, torch.Tensor)}
        if len(tensor_like) > 0:
            return strip_module_prefix(tensor_like)
    raise ValueError("Unsupported checkpoint format (cannot extract state_dict).")

@torch.no_grad()
def logits_to_pdanger(logits: torch.Tensor) -> torch.Tensor:
    # Supports: [B], [B,1] => sigmoid; [B,2] => softmax[:,1]
    if isinstance(logits, (tuple, list)):
        logits = logits[0]
    if logits.ndim == 1:
        return torch.sigmoid(logits)
    if logits.ndim == 2 and logits.size(1) == 1:
        return torch.sigmoid(logits[:, 0])
    if logits.ndim == 2 and logits.size(1) == 2:
        return F.softmax(logits, dim=-1)[:, 1]
    raise ValueError(f"Unexpected logits shape: {tuple(logits.shape)}")


In [ ]:
# IMPORTANT:
# Put train_esc50_binary.py into /content (same dir as this notebook)
# It must define: AudioFrontend, BCResNetBinary
from train_esc50_binary import AudioFrontend, BCResNetBinary

def infer_tau_from_filename(path: str) -> Optional[float]:
    base = os.path.basename(path).lower()
    m = re.search(r"bcresnet([0-9]+(?:\.[0-9]+)?)", base)
    return float(m.group(1)) if m else None

def load_bcresnet_binary(ckpt_path: str, device: str = DEVICE):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    state = extract_state_dict(ckpt)

    # tau must be known by constructor
    tau = infer_tau_from_filename(ckpt_path)
    if tau is None:
        raise ValueError(f"Cannot infer tau from filename: {ckpt_path}. Rename like bcresnet8.pt or pass a tau in filename.")

    # frontend (match your training defaults unless your train script encodes cfg elsewhere)
    frontend = AudioFrontend(
        sample_rate=SR,
        n_mels=40,
        win_ms=30.0,
        hop_ms=10.0,
        n_fft=1024,
        cmvn=True,
        specaug=False,
        freq_mask_param=10,
        time_mask_param=80,
        num_freq_masks=2,
        num_time_masks=2,
    ).to(device).eval()

    model = BCResNetBinary(tau=float(tau)).to(device).eval()

    missing, unexpected = model.load_state_dict(state, strict=False)
    if len(unexpected) > 0:
        print("[BCResNet] unexpected keys:", unexpected[:5], "...")
    if len(missing) > 0:
        print("[BCResNet] missing keys:", missing[:5], "...")

    return tau, frontend, model

@torch.no_grad()
def bcresnet_predict_pdanger(frontend, model, wave_32k_5s: torch.Tensor) -> torch.Tensor:
    # wave_32k_5s: (B, T) float
    feats = frontend(wave_32k_5s, augment=False).float()
    logits = model(feats)  # [B] logit
    return torch.sigmoid(logits)


In [ ]:
def build_mel_extractor(device: str = DEVICE):
    mel = AugmentMelSTFT(
        n_mels=128, sr=SR, win_length=800, hopsize=320, n_fft=1024,
        freqm=0, timem=0, fmin=0, fmax=None, fmin_aug_range=10, fmax_aug_range=2000
    ).to(device)
    mel.eval()
    return mel

@torch.no_grad()
def wave_to_mel_128(mel: AugmentMelSTFT, wave_b1t: torch.Tensor) -> torch.Tensor:
    # wave_b1t: (B,1,T)
    B, C, T = wave_b1t.shape
    x = wave_b1t.reshape(-1, T)      # (B, T)
    xm = mel(x)                      # (B, 128, frames)
    xm = xm.reshape(B, C, xm.shape[1], xm.shape[2])  # (B,1,128,frames)
    return xm

def load_efficientat_model(model_name: str, ckpt_path: str, num_classes: int, device: str = DEVICE):
    width = NAME_TO_WIDTH(model_name)
    if model_name.startswith("dymn"):
        model = get_dymn(width_mult=width, pretrained_name=None, pretrain_final_temp=1.0, num_classes=num_classes)
    else:
        model = get_mobilenet(width_mult=width, pretrained_name=None, head_type="mlp", se_dims="c", num_classes=num_classes)

    ckpt = torch.load(ckpt_path, map_location="cpu")
    state = extract_state_dict(ckpt)
    model.load_state_dict(state, strict=True)
    model.to(device).eval()
    return model

mel128 = build_mel_extractor()

mn_bin = load_efficientat_model(STAGE2_MN_MODEL_NAME, STAGE2_MN_CKPT, STAGE2_MN_NUM_CLASSES)
dymn_50 = load_efficientat_model(STAGE3_DYMN_MODEL_NAME, STAGE3_DYMN_CKPT, STAGE3_NUM_CLASSES)

print("Loaded MN binary + DyMN 50-class.")


/usr/local/lib/python3.12/dist-packages/torchvision/ops/misc.py:121: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(


MN(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
          (1): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(8, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(8, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Ba

In [ ]:
def macs_from_fvcore(model: torch.nn.Module, inp: torch.Tensor) -> Optional[float]:
    try:
        flops = FlopCountAnalysis(model, inp).total()
        # fvcore returns FLOPs; for convs, FLOPs ~ 2*MACs sometimes depending convention.
        # We standardize by reporting "Ops" as returned, but also compute an approximate MAC count.
        return float(flops)
    except Exception as e:
        print("[fvcore flopcount failed]:", str(e)[:200])
        return None

def macs_from_thop(model: torch.nn.Module, inp: torch.Tensor) -> Optional[float]:
    try:
        from thop import profile
        macs, params = profile(model, inputs=(inp,), verbose=False)
        return float(macs)
    except Exception as e:
        print("[thop profile failed]:", str(e)[:200])
        return None

def estimate_ops(model: torch.nn.Module, inp: torch.Tensor) -> Dict[str, Optional[float]]:
    model.eval()
    with torch.no_grad():
        fv = macs_from_fvcore(model, inp)
        th = macs_from_thop(model, inp)
    return {"fvcore_ops": fv, "thop_macs": th}

# Dummy inputs (match your inference shapes)
dummy_wave = torch.zeros(1, 1, CLIP_T, device=DEVICE)

dummy_mel = wave_to_mel_128(mel128, dummy_wave)
print("dummy_mel shape:", tuple(dummy_mel.shape))

# MN binary and DyMN 50 both take mel input and return (logits, _)
class WrapEffAT(torch.nn.Module):
    def __init__(self, m): super().__init__(); self.m=m
    def forward(self, x):
        out = self.m(x)
        return out[0] if isinstance(out, (tuple,list)) else out

mn_wrap = WrapEffAT(mn_bin).to(DEVICE)
dymn_wrap = WrapEffAT(dymn_50).to(DEVICE)

ops_mn = estimate_ops(mn_wrap, dummy_mel)
ops_dymn = estimate_ops(dymn_wrap, dummy_mel)

print("MN ops:", ops_mn)
print("DyMN ops:", ops_dymn)

# BC-ResNet: need its frontend output shape
# (load tau=8 ckpt as representative to get shape; ops are usually similar across tau)
_any_ckpt = list(BCRESNET_CKPTS.values())[0]
tau_tmp, frontend_tmp, bc_tmp = load_bcresnet_binary(_any_ckpt, device=DEVICE)

dummy_wave_bt = torch.zeros(1, CLIP_T, device=DEVICE)
with torch.no_grad():
    dummy_feats = frontend_tmp(dummy_wave_bt, augment=False).float()
print("dummy_feats shape (BCResNet input):", tuple(dummy_feats.shape))

ops_bc = estimate_ops(bc_tmp, dummy_feats)
print("BCResNet ops:", ops_bc)

# Pick one consistent MAC proxy:
# Prefer thop_macs (more directly MAC-ish), fallback to fvcore_ops.
def pick_macs(d: Dict[str, Optional[float]]) -> float:
    if d["thop_macs"] is not None:
        return float(d["thop_macs"])
    if d["fvcore_ops"] is not None:
        return float(d["fvcore_ops"])
    return float("nan")

MACS_STAGE2_MN = pick_macs(ops_mn)
MACS_STAGE3_DYMN = pick_macs(ops_dymn)
MACS_STAGE2_BC   = pick_macs(ops_bc)

print("MACS proxies:")
print("  Stage2 MN:", MACS_STAGE2_MN)
print("  Stage2 BC:", MACS_STAGE2_BC)
print("  Stage3 DyMN:", MACS_STAGE3_DYMN)


/usr/local/lib/python3.12/dist-packages/torch/functional.py:681: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:875.)
  return _VF.stft(  # type: ignore[attr-defined]
/content/EfficientAT/models/preprocess.py:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


dummy_mel shape: (1, 1, 128, 500)


m.layers.0.context_gen.pool_f, m.layers.0.context_gen.pool_t, m.layers.0.exp_act, m.layers.0.exp_conv, m.layers.10.context_gen.pool_f, m.layers.10.context_gen.pool_t, m.layers.11.context_gen.pool_f, m.layers.11.context_gen.pool_t, m.layers.13.context_gen.pool_f, m.layers.13.context_gen.pool_t, m.layers.14.context_gen.pool_f, m.layers.14.context_gen.pool_t, m.layers.2.context_gen.pool_f, m.layers.2.context_gen.pool_t, m.layers.4.context_gen.pool_f, m.layers.4.context_gen.pool_t, m.layers.5.context_gen.pool_f, m.layers.5.context_gen.pool_t, m.layers.7.context_gen.pool_f, m.layers.7.context_gen.pool_t, m.layers.8.context_gen.pool_f, m.layers.8.context_gen.pool_t, m.layers.9.context_gen.pool_f, m.layers.9.context_gen.pool_t


MN ops: {'fvcore_ops': 60623616.0, 'thop_macs': 65430656.0}
DyMN ops: {'fvcore_ops': 312468160.0, 'thop_macs': 58037024.0}
dummy_feats shape (BCResNet input): (1, 1, 40, 501)


BCResNet ops: {'fvcore_ops': 15448868.0, 'thop_macs': 18022036.0}
MACS proxies:
  Stage2 MN: 65430656.0
  Stage2 BC: 18022036.0
  Stage3 DyMN: 58037024.0


In [ ]:
@dataclass
class DSPGateConfig:
    sr: int = 16000
    frame_s: float = 0.5
    hop_s: float = 0.05
    n_fft: int = 512

    # noise tracking
    ema_alpha: float = 0.02

    # variant knobs
    delta_db: float = 12.0         # energy over noise floor
    flux_thr: float = 0.35         # normalized spectral flux
    band_thr_db: float = 10.0      # band energy over noise floor
    cooldown_s: float = 0.5        # avoid repeated triggers
    hysteresis_frames: int = 2

    # "sleep mode"
    sleep_db: float = -45.0        # if long-term energy below this, relax thresholds
    sleep_delta_db: float = 8.0
    sleep_flux_thr: float = 0.25

def downsample_to_16k(wave_32k: np.ndarray) -> np.ndarray:
    w = torch.from_numpy(wave_32k)
    w16 = torchaudio.functional.resample(w, SR, 16000).numpy().astype(np.float32)
    return w16

def frame_rms_db(x: np.ndarray, eps=1e-8) -> float:
    rms = np.sqrt(np.mean(x*x) + eps)
    return 20.0*np.log10(rms + eps)

def spectral_flux(prev_mag: np.ndarray, mag: np.ndarray, eps=1e-8) -> float:
    # normalized L1 positive changes
    diff = np.maximum(mag - prev_mag, 0.0)
    return float(np.sum(diff) / (np.sum(mag) + eps))

def stft_mag(x: np.ndarray, n_fft: int) -> np.ndarray:
    win = np.hanning(len(x)).astype(np.float32)
    X = np.fft.rfft(x * win, n=n_fft)
    return np.abs(X).astype(np.float32)

def band_energies(mag: np.ndarray, sr: int, n_fft: int) -> np.ndarray:
    # 4 broad bands: low/mid/high/very-high
    freqs = np.fft.rfftfreq(n_fft, d=1/sr)
    bands = [(0, 300), (300, 1200), (1200, 3000), (3000, sr/2)]
    out = []
    for lo, hi in bands:
        idx = np.where((freqs >= lo) & (freqs < hi))[0]
        out.append(float(np.mean(mag[idx] + 1e-8)))
    return np.array(out, dtype=np.float32)

class DSPGate:
    """
    Returns triggers on hop grid. Designed to be cheap and high-recall.
    """
    def __init__(self, cfg: DSPGateConfig, variant: str):
        assert variant in ["A_energy", "B_energy_flux", "C_sleep_aware"]
        self.cfg = cfg
        self.variant = variant

        self.noise_db = None
        self.noise_band = None
        self.prev_mag = None

        self.cooldown_frames = int(round(cfg.cooldown_s / cfg.hop_s))
        self.cooldown_left = 0
        self.on_count = 0

        # long-term for sleep mode
        self.long_rms_ema = None

    def reset(self):
        self.noise_db = None
        self.noise_band = None
        self.prev_mag = None
        self.cooldown_left = 0
        self.on_count = 0
        self.long_rms_ema = None

    def step(self, frame: np.ndarray) -> bool:
        cfg = self.cfg

        # update long RMS (for sleep mode)
        rms_db = frame_rms_db(frame)
        if self.long_rms_ema is None:
            self.long_rms_ema = rms_db
        else:
            self.long_rms_ema = 0.995*self.long_rms_ema + 0.005*rms_db

        # STFT magnitude + bands
        mag = stft_mag(frame, cfg.n_fft)
        bands = band_energies(mag, cfg.sr, cfg.n_fft)

        # init noise trackers
        if self.noise_db is None:
            self.noise_db = rms_db
            self.noise_band = bands.copy()
            self.prev_mag = mag.copy()
            return False

        # EMA update (assume most frames are non-event)
        self.noise_db = (1-cfg.ema_alpha)*self.noise_db + cfg.ema_alpha*rms_db
        self.noise_band = (1-cfg.ema_alpha)*self.noise_band + cfg.ema_alpha*bands

        # thresholds (sleep-aware)
        delta_db = cfg.delta_db
        flux_thr = cfg.flux_thr
        if self.variant == "C_sleep_aware" and self.long_rms_ema < cfg.sleep_db:
            delta_db = cfg.sleep_delta_db
            flux_thr = cfg.sleep_flux_thr

        # compute gates
        energy_hit = (rms_db - self.noise_db) > delta_db

        flux = spectral_flux(self.prev_mag, mag)
        flux_hit = flux > flux_thr

        # band hit: any band rises a lot over its noise estimate (in dB-ish)
        band_ratio = (bands + 1e-6) / (self.noise_band + 1e-6)
        band_hit = np.max(20*np.log10(band_ratio)) > cfg.band_thr_db

        self.prev_mag = mag

        if self.cooldown_left > 0:
            self.cooldown_left -= 1
            return False

        if self.variant == "A_energy":
            trig = energy_hit
        elif self.variant == "B_energy_flux":
            trig = energy_hit or (flux_hit and band_hit)
        else:  # C_sleep_aware
            # sleep-aware is more willing to trigger on flux/band even at low RMS
            trig = energy_hit or flux_hit or band_hit

        # hysteresis: require consecutive hits to declare event (reduces single-frame noise)
        if trig:
            self.on_count += 1
        else:
            self.on_count = max(0, self.on_count - 1)

        if self.on_count >= cfg.hysteresis_frames:
            self.on_count = 0
            self.cooldown_left = self.cooldown_frames
            return True

        return False

    def run(self, wave_32k: np.ndarray) -> Tuple[np.ndarray, float]:
        """
        Returns:
          triggers: boolean array for each hop
          sec_per_hop: hop duration seconds
        """
        cfg = self.cfg
        w16 = downsample_to_16k(wave_32k)

        frame_len = int(cfg.frame_s * cfg.sr)
        hop_len = int(cfg.hop_s * cfg.sr)

        n_hops = 1 + max(0, (len(w16) - frame_len) // hop_len)
        triggers = np.zeros(n_hops, dtype=np.bool_)

        self.reset()
        for i in range(n_hops):
            s = i*hop_len
            frame = w16[s:s+frame_len]
            if len(frame) < frame_len:
                frame = np.pad(frame, (0, frame_len - len(frame)))
            triggers[i] = self.step(frame)

        return triggers, cfg.hop_s

# Create 3 gate variants
gate_cfg = DSPGateConfig()
DSP_GATES = {
    "S1_A_energy": DSPGate(gate_cfg, "A_energy"),
    "S1_B_energy_flux": DSPGate(gate_cfg, "B_energy_flux"),
    "S1_C_sleep_aware": DSPGate(gate_cfg, "C_sleep_aware"),
}

print("Stage 1 DSP gates ready:", list(DSP_GATES.keys()))


Stage 1 DSP gates ready: ['S1_A_energy', 'S1_B_energy_flux', 'S1_C_sleep_aware']


In [ ]:
# --- Explicit categorization ---
AMBIENT_CATS = {
    "rain", "sea_waves", "crickets", "chirping_birds", "wind",
    "clock_tick", "washing_machine", "airplane", "helicopter",
    "engine", "train", "church_bells", "water_drops",
}

# Everything not emergency and not ambient becomes "event-like safe"
ALL_CATS = set(df["category"].unique())
SAFE_EVENT_CATS = sorted(list(ALL_CATS - EMERGENCY_CATS - AMBIENT_CATS))

def butter_lowpass(x: np.ndarray, sr: int, cutoff_hz: float) -> np.ndarray:
    if cutoff_hz >= sr/2:
        return x
    b, a = signal.butter(2, cutoff_hz/(sr/2), btype="low")
    return signal.lfilter(b, a, x).astype(np.float32)

def simple_reverb(x: np.ndarray, sr: int, rt60: float = 0.25) -> np.ndarray:
    # lightweight synthetic RIR: exponential decay noise
    rir_len = int(sr * min(0.5, max(0.05, rt60*1.2)))
    t = np.arange(rir_len, dtype=np.float32) / sr
    rir = (np.random.randn(rir_len).astype(np.float32) * np.exp(-t / max(1e-3, rt60))).astype(np.float32)
    rir[0] += 1.0
    y = signal.fftconvolve(x, rir, mode="full")[:len(x)].astype(np.float32)
    y = y / (np.max(np.abs(y)) + 1e-8) * (np.max(np.abs(x)) + 1e-8)
    return y

def apply_event_effects(x: np.ndarray, sr: int, kind: str, rng: np.random.RandomState) -> np.ndarray:
    """
    kind: "bg" | "safe" | "emergency"
    """
    y = x.copy()

    # mild reverb sometimes
    if rng.rand() < (0.15 if kind != "bg" else 0.05):
        rt60 = rng.uniform(0.12, 0.35)
        y = simple_reverb(y, sr, rt60=rt60)

    # occlusion / outside-room lowpass sometimes (important for "quiet but real danger")
    if rng.rand() < (0.25 if kind == "emergency" else 0.12):
        cutoff = rng.uniform(800, 3000)
        y = butter_lowpass(y, sr, cutoff_hz=cutoff)

    return y.astype(np.float32)

def set_snr(signal_x: np.ndarray, noise_x: np.ndarray, snr_db: float) -> np.ndarray:
    """
    Returns scaled signal to achieve desired SNR wrt noise (both same length).
    """
    eps = 1e-8
    ps = np.mean(signal_x**2) + eps
    pn = np.mean(noise_x**2) + eps
    desired_ps = pn * (10**(snr_db/10))
    scale = math.sqrt(desired_ps / ps)
    return (signal_x * scale).astype(np.float32)

@dataclass
class ScenarioConfig:
    name: str
    duration_s: float
    bg_level_db: Tuple[float, float]             # base loudness range
    safe_events_per_min: Tuple[float, float]
    emerg_events_per_min: Tuple[float, float]
    emerg_snr_db: Tuple[float, float]            # relative to background
    safe_snr_db: Tuple[float, float]
    overlap_prob: float
    night_mode: bool = False

SCENARIOS = [
    ScenarioConfig(
        name="quiet_home_day",
        duration_s=10*60,
        bg_level_db=(-38, -30),
        safe_events_per_min=(1.0, 2.5),
        emerg_events_per_min=(0.15, 0.35),
        emerg_snr_db=(-2, 14),
        safe_snr_db=(0, 16),
        overlap_prob=0.10,
        night_mode=False,
    ),
    ScenarioConfig(
        name="sleep_night",
        duration_s=10*60,
        bg_level_db=(-55, -45),
        safe_events_per_min=(0.2, 0.8),
        emerg_events_per_min=(0.08, 0.22),
        emerg_snr_db=(-8, 10),   # more quiet/far emergencies
        safe_snr_db=(-4, 10),
        overlap_prob=0.05,
        night_mode=True,
    ),
    ScenarioConfig(
        name="busy_indoor",
        duration_s=10*60,
        bg_level_db=(-30, -22),
        safe_events_per_min=(2.0, 4.0),
        emerg_events_per_min=(0.12, 0.30),
        emerg_snr_db=(-5, 12),
        safe_snr_db=(0, 12),
        overlap_prob=0.20,
        night_mode=False,
    ),
    ScenarioConfig(
        name="kitchen_like",
        duration_s=10*60,
        bg_level_db=(-33, -24),
        safe_events_per_min=(1.5, 3.0),
        emerg_events_per_min=(0.12, 0.28),
        emerg_snr_db=(-4, 12),
        safe_snr_db=(0, 14),
        overlap_prob=0.18,
        night_mode=False,
    ),
    ScenarioConfig(
        name="outdoor_park",
        duration_s=10*60,
        bg_level_db=(-36, -28),
        safe_events_per_min=(1.0, 2.5),
        emerg_events_per_min=(0.10, 0.25),
        emerg_snr_db=(-6, 12),
        safe_snr_db=(-2, 14),
        overlap_prob=0.12,
        night_mode=False,
    ),
    ScenarioConfig(
        name="commute_like",
        duration_s=10*60,
        bg_level_db=(-28, -20),
        safe_events_per_min=(2.0, 4.5),
        emerg_events_per_min=(0.15, 0.35),
        emerg_snr_db=(-5, 10),
        safe_snr_db=(0, 10),
        overlap_prob=0.25,
        night_mode=False,
    ),
]

def sample_clip_by_category(category: str, rng: np.random.RandomState) -> np.ndarray:
    rows = df[df["category"] == category]
    idx = int(rng.choice(rows.index.values))
    path = resolve_audio_path(ESC50_32K_ROOT, df.loc[idx, "filename"])
    return load_audio_32k_5s(path)

def build_background(duration_s: float, rng: np.random.RandomState, level_db: float) -> np.ndarray:
    T = int(SR * duration_s)
    y = np.zeros(T, dtype=np.float32)

    # base: low gaussian noise
    noise = rng.randn(T).astype(np.float32)
    noise = noise / (np.std(noise) + 1e-8)
    noise_gain = 10**(level_db/20)
    y += noise * noise_gain * 0.2

    # add tiled ambient clips
    pos = 0
    amb_list = list(AMBIENT_CATS & set(df["category"].unique()))
    if len(amb_list) == 0:
        return np.clip(y, -1, 1)

    while pos < T:
        cat = rng.choice(amb_list)
        clip = sample_clip_by_category(cat, rng)
        clip = apply_event_effects(clip, SR, "bg", rng)

        # random gain around background
        g_db = rng.uniform(level_db - 6, level_db + 3)
        g = 10**(g_db/20)
        clip = clip * g

        end = min(T, pos + len(clip))
        y[pos:end] += clip[:end-pos]
        pos = end

    return np.clip(y, -1.0, 1.0)

def poisson_event_times(rate_per_min: float, duration_s: float, rng: np.random.RandomState) -> List[float]:
    rate_per_s = rate_per_min / 60.0
    t = 0.0
    out = []
    while t < duration_s:
        # exponential inter-arrival
        dt = rng.exponential(1.0 / max(1e-6, rate_per_s))
        t += dt
        if t < duration_s:
            out.append(float(t))
    return out

def overlay_events(base: np.ndarray, scen: ScenarioConfig, rng: np.random.RandomState) -> Tuple[np.ndarray, List[Dict[str, Any]]]:
    duration_s = scen.duration_s
    y = base.copy()
    ann = []

    # choose categories pool
    emerg_pool = sorted(list(EMERGENCY_CATS))
    safe_pool = SAFE_EVENT_CATS

    safe_rate = rng.uniform(*scen.safe_events_per_min)
    emerg_rate = rng.uniform(*scen.emerg_events_per_min)

    safe_times = poisson_event_times(safe_rate, duration_s, rng)
    emerg_times = poisson_event_times(emerg_rate, duration_s, rng)

    def do_one_event(t_s: float, kind: str):
        cat = rng.choice(emerg_pool) if kind == "emergency" else rng.choice(safe_pool)
        clip = sample_clip_by_category(cat, rng)
        clip = apply_event_effects(clip, SR, kind, rng)

        # sometimes overlap by shifting start earlier (burstiness)
        if rng.rand() < scen.overlap_prob:
            t_s = max(0.0, t_s - rng.uniform(0.1, 1.5))

        start = int(t_s * SR)
        end = min(len(y), start + len(clip))
        clip = clip[:end-start]

        # set SNR relative to local background segment
        bg_seg = y[start:end].copy()
        if len(bg_seg) < 32:
            return

        snr_db = rng.uniform(*(scen.emerg_snr_db if kind=="emergency" else scen.safe_snr_db))
        clip = set_snr(clip, bg_seg, snr_db)

        y[start:end] = np.clip(y[start:end] + clip, -1.0, 1.0)

        ann.append({
            "t_start": start / SR,
            "t_end": end / SR,
            "category": str(cat),
            "kind": kind,
            "snr_db": float(snr_db),
        })

    for t_s in safe_times:
        do_one_event(t_s, "safe")
    for t_s in emerg_times:
        do_one_event(t_s, "emergency")

    ann = sorted(ann, key=lambda d: d["t_start"])
    return y, ann

def build_soundscape(scen: ScenarioConfig, seed: int) -> Tuple[np.ndarray, List[Dict[str, Any]]]:
    rng = np.random.RandomState(seed)
    bg_level = rng.uniform(*scen.bg_level_db)
    base = build_background(scen.duration_s, rng, level_db=bg_level)
    y, ann = overlay_events(base, scen, rng)
    return y, ann

print("Synthetic soundscapes configured:", [s.name for s in SCENARIOS])


Synthetic soundscapes configured: ['quiet_home_day', 'sleep_night', 'busy_indoor', 'kitchen_like', 'outdoor_park', 'commute_like']


In [ ]:
@dataclass
class EvalConfig:
    decision_hop_s: float = 0.5     # how often we make a final decision (and compute EF1 windows)
    label_win_s: float = 1.0        # ground-truth window length for EF1 labeling
    stage2_gate: float = 0.5        # p(danger) threshold to invoke stage3
    stage2_uncert_lo: Optional[float] = None  # e.g. 0.4
    stage2_uncert_hi: Optional[float] = None  # e.g. 0.6

EVAL_CFG = EvalConfig()

def get_trailing_5s(wave: np.ndarray, t_end_s: float) -> np.ndarray:
    end = int(t_end_s * SR)
    start = end - CLIP_T
    if start < 0:
        seg = np.pad(wave[:end], (abs(start), 0))
    else:
        seg = wave[start:end]
    if len(seg) < CLIP_T:
        seg = np.pad(seg, (0, CLIP_T - len(seg)))
    return seg.astype(np.float32)

def build_gt_emergency_windows(ann: List[Dict[str, Any]], duration_s: float, hop_s: float, label_win_s: float) -> np.ndarray:
    times = np.arange(hop_s, duration_s + 1e-6, hop_s)
    y = np.zeros(len(times), dtype=np.int64)

    # mark window positive if overlaps any emergency event within [t-label_win, t]
    for i, t in enumerate(times):
        w0, w1 = t - label_win_s, t
        for a in ann:
            if a["kind"] == "emergency":
                if not (a["t_end"] <= w0 or a["t_start"] >= w1):
                    y[i] = 1
                    break
    return y

@torch.no_grad()
def stage3_predict_target(wave_5s: torch.Tensor) -> torch.Tensor:
    # wave_5s: (B,1,T)
    xm = wave_to_mel_128(mel128, wave_5s)
    logits = dymn_50(xm)[0]  # (B,50)
    return logits.argmax(dim=-1)

@torch.no_grad()
def stage2_mn_pdanger(wave_5s: torch.Tensor) -> torch.Tensor:
    xm = wave_to_mel_128(mel128, wave_5s)
    logits = mn_bin(xm)[0]   # (B,2)
    return logits_to_pdanger(logits)

def stage2_policy(p: float, cfg: EvalConfig) -> bool:
    if cfg.stage2_uncert_lo is not None and cfg.stage2_uncert_hi is not None:
        # escalate if danger OR uncertain (recall-friendly)
        return (p >= cfg.stage2_gate) or (cfg.stage2_uncert_lo <= p <= cfg.stage2_uncert_hi)
    return (p >= cfg.stage2_gate)


In [ ]:
def f1_precision_recall(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, float, float]:
    tp = int(((y_true==1) & (y_pred==1)).sum())
    fp = int(((y_true==0) & (y_pred==1)).sum())
    fn = int(((y_true==1) & (y_pred==0)).sum())
    prec = tp / max(1, tp+fp)
    rec  = tp / max(1, tp+fn)
    f1 = 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)
    return float(f1), float(prec), float(rec)

def auroc_from_scores(y_true: np.ndarray, score: np.ndarray) -> float:
    # simple AUROC without sklearn
    # rank-based (Mann–Whitney U)
    y_true = y_true.astype(np.int64)
    score = score.astype(np.float64)
    pos = score[y_true==1]
    neg = score[y_true==0]
    if len(pos)==0 or len(neg)==0:
        return float("nan")
    # compute probability pos>neg + 0.5 ties
    count = 0.0
    for p in pos:
        count += np.sum(p > neg) + 0.5*np.sum(p == neg)
    return float(count / (len(pos)*len(neg)))

def event_ttd(ann: List[Dict[str, Any]], decision_times: np.ndarray, pred_emergency: np.ndarray, tol_s: float = 2.0) -> float:
    """
    Time-to-detection: for each emergency event, find earliest predicted emergency in [start, end+tol],
    return median delay (sec). If no detections, return NaN.
    """
    delays = []
    for a in ann:
        if a["kind"] != "emergency":
            continue
        start = a["t_start"]
        end = a["t_end"] + tol_s
        idx = np.where((decision_times >= start) & (decision_times <= end) & (pred_emergency==1))[0]
        if len(idx) > 0:
            delays.append(float(decision_times[idx[0]] - start))
    if len(delays) == 0:
        return float("nan")
    return float(np.median(delays))

def compute_cavg_macs(duration_s: float,
                      n_stage1_calls: int,
                      n_stage2_calls: int,
                      n_stage3_calls: int,
                      mac_stage1_per_call: float,
                      mac_stage2_per_call: float,
                      mac_stage3_per_call: float) -> float:
    total = (n_stage1_calls*mac_stage1_per_call +
             n_stage2_calls*mac_stage2_per_call +
             n_stage3_calls*mac_stage3_per_call)
    return float(total / max(1e-6, duration_s))

# Stage1 MAC proxy:
# We'll treat Stage1 as FFT-ish work per hop (very small vs NN), but still nonzero.
# Rough proxy: k * n_fft*log2(n_fft) per hop; use k=8 to be conservative.
def stage1_mac_proxy(cfg: DSPGateConfig) -> float:
    n_fft = cfg.n_fft
    per = 8.0 * n_fft * math.log2(n_fft)
    return float(per)

MAC_STAGE1_PER_CALL = stage1_mac_proxy(gate_cfg)
print("Stage1 proxy MAC/call:", MAC_STAGE1_PER_CALL)


Stage1 proxy MAC/call: 36864.0


In [ ]:
@torch.no_grad()
def run_single_clip_stage3(df_eval: pd.DataFrame, batch_size: int = 64) -> Dict[str, float]:
    # Stage3 always-on over individual ESC-50 clips (50-way)
    y_true_em = df_eval["is_emergency"].values.astype(np.int64)
    probs_em = []
    preds_em = []

    # optional: top1 acc
    y_true_t = df_eval["target"].values.astype(np.int64)
    y_pred_t = []

    for i in tqdm(range(0, len(df_eval), batch_size), desc="Stage3 single-clip"):
        sub = df_eval.iloc[i:i+batch_size]
        waves = []
        for fn in sub["filename"].values:
            path = resolve_audio_path(ESC50_32K_ROOT, fn)
            waves.append(load_audio_32k_5s(path))
        wave = torch.from_numpy(np.stack(waves))[:, None, :].to(DEVICE)  # (B,1,T)

        xm = wave_to_mel_128(mel128, wave)
        logits = dymn_50(xm)[0]  # (B,50)
        p = F.softmax(logits, dim=-1).detach().cpu().numpy()
        pred_t = p.argmax(axis=-1)

        # emergency score = sum probs over emergency targets
        em_targets = [cat_to_target[c] for c in EMERGENCY_CATS]
        p_em = p[:, em_targets].sum(axis=-1)
        pred_em = (p_em >= 0.5).astype(np.int64)

        probs_em.append(p_em)
        preds_em.append(pred_em)
        y_pred_t.append(pred_t)

    probs_em = np.concatenate(probs_em)
    preds_em = np.concatenate(preds_em)
    y_pred_t = np.concatenate(y_pred_t)

    ef1, prec, rec = f1_precision_recall(y_true_em, preds_em)
    auroc = auroc_from_scores(y_true_em, probs_em)
    acc = float((y_pred_t == y_true_t).mean())

    # compute (always-on): one stage3 call per clip
    cavg = compute_cavg_macs(
        duration_s=len(df_eval)*CLIP_S,
        n_stage1_calls=0,
        n_stage2_calls=0,
        n_stage3_calls=len(df_eval),
        mac_stage1_per_call=MAC_STAGE1_PER_CALL,
        mac_stage2_per_call=0.0,
        mac_stage3_per_call=MACS_STAGE3_DYMN,
    )
    eui = ef1 / max(1e-9, cavg)

    return {
        "EF1": ef1, "Prec": prec, "Rec": rec, "AUROC": auroc,
        "Top1_Acc": acc,
        "Cavg_MACs_per_s": cavg,
        "EUI": eui,
        "Stage3_calls": int(len(df_eval)),
    }

@torch.no_grad()
def run_single_clip_stage2_stage3(df_eval: pd.DataFrame,
                                 stage2_name: str,
                                 stage2_type: str,
                                 bc_frontend=None,
                                 bc_model=None,
                                 batch_size: int = 64,
                                 cfg: EvalConfig = EVAL_CFG) -> Dict[str, float]:
    """
    stage2_type: "mn" or "bc"
    pipeline: Stage2 always, Stage3 only if Stage2 escalates
    output emergency = (Stage3 in E) if invoked else 0
    """
    y_true_em = df_eval["is_emergency"].values.astype(np.int64)

    preds_em = []
    scores_em = []
    n_stage3 = 0

    for i in tqdm(range(0, len(df_eval), batch_size), desc=f"S2+S3 single-clip ({stage2_name})"):
        sub = df_eval.iloc[i:i+batch_size]
        waves = []
        for fn in sub["filename"].values:
            path = resolve_audio_path(ESC50_32K_ROOT, fn)
            waves.append(load_audio_32k_5s(path))
        wave = torch.from_numpy(np.stack(waves))[:, None, :].to(DEVICE)  # (B,1,T)

        # Stage2 p(danger)
        if stage2_type == "mn":
            p_d = stage2_mn_pdanger(wave).detach().cpu().numpy()
        else:
            p_d = bcresnet_predict_pdanger(bc_frontend, bc_model, wave[:,0,:]).detach().cpu().numpy()

        # decide escalation per item
        esc = np.array([stage2_policy(float(p), cfg) for p in p_d], dtype=np.bool_)

        # Stage3 only for escalated
        pred_em = np.zeros(len(sub), dtype=np.int64)
        score_em = np.zeros(len(sub), dtype=np.float32)

        if esc.any():
            idx = np.where(esc)[0]
            wave_esc = wave[idx]
            pred_t = stage3_predict_target(wave_esc).detach().cpu().numpy().astype(np.int64)
            pred_cat = np.array([target_to_cat[int(t)] for t in pred_t], dtype=object)
            pred_em[idx] = np.array([1 if c in EMERGENCY_CATS else 0 for c in pred_cat], dtype=np.int64)
            score_em[idx] = p_d[idx]  # use stage2 probability as "emergency-likelihood" score proxy
            n_stage3 += int(len(idx))

        preds_em.append(pred_em)
        scores_em.append(score_em)

    preds_em = np.concatenate(preds_em)
    scores_em = np.concatenate(scores_em)

    ef1, prec, rec = f1_precision_recall(y_true_em, preds_em)
    auroc = auroc_from_scores(y_true_em, scores_em)

    duration = len(df_eval)*CLIP_S
    cavg = compute_cavg_macs(
        duration_s=duration,
        n_stage1_calls=0,
        n_stage2_calls=len(df_eval),
        n_stage3_calls=n_stage3,
        mac_stage1_per_call=MAC_STAGE1_PER_CALL,
        mac_stage2_per_call=(MACS_STAGE2_MN if stage2_type=="mn" else MACS_STAGE2_BC),
        mac_stage3_per_call=MACS_STAGE3_DYMN,
    )
    eui = ef1 / max(1e-9, cavg)

    return {
        "EF1": ef1, "Prec": prec, "Rec": rec, "AUROC": auroc,
        "Cavg_MACs_per_s": cavg,
        "EUI": eui,
        "Stage2_calls": int(len(df_eval)),
        "Stage3_calls": int(n_stage3),
        "Stage2_name": stage2_name,
    }

@torch.no_grad()
def run_soundscape_pipeline(wave: np.ndarray,
                           ann: List[Dict[str, Any]],
                           pipeline: str,
                           stage1_gate: Optional[DSPGate],
                           stage2_name: Optional[str],
                           stage2_type: Optional[str],
                           bc_frontend=None,
                           bc_model=None,
                           cfg: EvalConfig = EVAL_CFG) -> Dict[str, float]:
    """
    pipelines:
      "S3"                    : Stage3 on every decision hop (always-on)
      "S1+S3"                 : Stage1 decides whether to run Stage3
      "S2+S3"                 : Stage2 always, Stage3 conditional
      "S1+S2+S3"              : Stage1 gate -> Stage2 -> Stage3 conditional
    """
    duration_s = len(wave)/SR
    decision_times = np.arange(cfg.decision_hop_s, duration_s + 1e-6, cfg.decision_hop_s)
    y_true = build_gt_emergency_windows(ann, duration_s, cfg.decision_hop_s, cfg.label_win_s)

    pred = np.zeros(len(decision_times), dtype=np.int64)
    score = np.zeros(len(decision_times), dtype=np.float32)

    # Stage1 triggers on its own hop grid; map into decision grid if used
    stage1_trig = None
    if stage1_gate is not None:
        trig_hops, s1_hop = stage1_gate.run(wave)
        # convert stage1 trigger times into seconds
        trig_times = (np.where(trig_hops)[0] * s1_hop).astype(np.float32)
        stage1_trig = trig_times

    n_s1 = len(decision_times) if stage1_gate is not None else 0
    n_s2 = 0
    n_s3 = 0

    # quick helper: is there a stage1 trigger in last label window?
    def s1_fired_recent(t: float) -> bool:
        if stage1_trig is None or len(stage1_trig)==0:
            return False
        w0, w1 = t - cfg.label_win_s, t
        # any trigger in [w0, w1]
        i0 = np.searchsorted(stage1_trig, w0, side="left")
        i1 = np.searchsorted(stage1_trig, w1, side="right")
        return i1 > i0

    for i, t in enumerate(decision_times):
        # decide which stages to run
        run_s3 = False
        run_s2 = False

        if pipeline == "S3":
            run_s3 = True
        elif pipeline == "S1+S3":
            run_s3 = s1_fired_recent(float(t))
        elif pipeline == "S2+S3":
            run_s2 = True
        elif pipeline == "S1+S2+S3":
            run_s2 = s1_fired_recent(float(t))
        else:
            raise ValueError("Unknown pipeline")

        # extract trailing 5s
        seg = get_trailing_5s(wave, float(t))
        wave_t = torch.from_numpy(seg)[None, None, :].to(DEVICE)

        # Stage2
        if run_s2:
            n_s2 += 1
            if stage2_type == "mn":
                p_d = float(stage2_mn_pdanger(wave_t).item())
            else:
                p_d = float(bcresnet_predict_pdanger(bc_frontend, bc_model, wave_t[:,0,:]).item())
            score[i] = p_d
            run_s3 = stage2_policy(p_d, cfg)
        else:
            score[i] = 0.0

        # Stage3
        if run_s3:
            n_s3 += 1
            pred_t = int(stage3_predict_target(wave_t).item())
            pred_cat = target_to_cat[pred_t]
            pred[i] = 1 if pred_cat in EMERGENCY_CATS else 0
            if pipeline == "S3" or pipeline == "S1+S3":
                # if no stage2, use emergency score from stage3 probs (more informative)
                xm = wave_to_mel_128(mel128, wave_t)
                logits = dymn_50(xm)[0]
                p = F.softmax(logits, dim=-1)[0].detach().cpu().numpy()
                em_targets = [cat_to_target[c] for c in EMERGENCY_CATS]
                score[i] = float(p[em_targets].sum())

    ef1, prec, rec = f1_precision_recall(y_true, pred)
    auroc = auroc_from_scores(y_true, score)

    # TR / DER
    tr = float(np.mean([s1_fired_recent(float(t)) for t in decision_times])) if stage1_gate is not None else float("nan")
    der = float(n_s3 / max(1, n_s1)) if stage1_gate is not None else float(n_s3 / max(1, len(decision_times)))

    # TTD (event-level)
    ttd = event_ttd(ann, decision_times, pred, tol_s=2.0)

    # Compute (MACs/s)
    mac_s2 = MACS_STAGE2_MN if (stage2_type=="mn") else MACS_STAGE2_BC
    cavg = compute_cavg_macs(
        duration_s=duration_s,
        n_stage1_calls=n_s1,
        n_stage2_calls=n_s2,
        n_stage3_calls=n_s3,
        mac_stage1_per_call=MAC_STAGE1_PER_CALL,
        mac_stage2_per_call=mac_s2 if stage2_type is not None else 0.0,
        mac_stage3_per_call=MACS_STAGE3_DYMN,
    )
    eui = ef1 / max(1e-9, cavg)

    return {
        "EF1": ef1, "Prec": prec, "Rec": rec, "AUROC": auroc,
        "TR": tr, "DER": der, "TTD_sec": ttd,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": int(n_s1), "Stage2_calls": int(n_s2), "Stage3_calls": int(n_s3),
        "pipeline": pipeline,
        "stage2_name": stage2_name if stage2_name else "",
        "stage1_name": "" if stage1_gate is None else "DSP",
    }


In [ ]:
# ============================================================
# SMOKE RUN
# - Runs end-to-end assuming your main notebook cells already defined:
#   df, ESC50_32K_ROOT, resolve_audio_path, load_audio_32k_5s, SR, CLIP_T, CLIP_S
#   EMERGENCY_CATS, target_to_cat, cat_to_target
#   build_soundscape, SCENARIOS, ScenarioConfig
#   stage2_mn_pdanger, stage3_predict_target
#   BCRESNET_CKPTS, load_bcresnet_binary, bcresnet_predict_pdanger
#   DSPGateConfig, DSPGate
#   MAC_STAGE1_PER_CALL, MACS_STAGE2_MN, MACS_STAGE2_BC, MACS_STAGE3_DYMN, compute_cavg_macs
#   mel128, wave_to_mel_128, dymn_50 (for S3 score)
# - Smoke:
#   (1) Stage3 on 32 clips
#   (2) Stage2+Stage3 on 32 clips (MN + BC tau8)
#   (3) 1x 60s soundscape (forced >=1 emergency @ high SNR) with 4 pipelines:
#       S3, S1+S3, S2+S3(MN), S1+S2+S3(MN)
# - Saves: OUT_DIR/smoke_results.csv
# ============================================================

import os, math, warnings
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

# -----------------------------
# Silence EfficientAT warning (optional)
# -----------------------------
warnings.filterwarnings("ignore", category=FutureWarning, message=".*autocast.*deprecated.*")

# -----------------------------
# Safety checks (must exist)
# -----------------------------
needed = [
    "df", "ESC50_32K_ROOT", "resolve_audio_path", "load_audio_32k_5s", "SR", "CLIP_T", "CLIP_S",
    "EMERGENCY_CATS", "target_to_cat", "cat_to_target",
    "SCENARIOS", "ScenarioConfig", "build_soundscape",
    "BCRESNET_CKPTS", "load_bcresnet_binary", "bcresnet_predict_pdanger",
    "stage2_mn_pdanger", "stage3_predict_target",
    "DSPGateConfig", "DSPGate",
    "OUT_DIR", "DEVICE",
    "MAC_STAGE1_PER_CALL", "MACS_STAGE2_MN", "MACS_STAGE2_BC", "MACS_STAGE3_DYMN", "compute_cavg_macs",
    "mel128", "wave_to_mel_128", "dymn_50"
]
missing = [x for x in needed if x not in globals()]
assert not missing, f"Missing definitions: {missing}"

# -----------------------------
# Smoke config
# -----------------------------
SMOKE_N_CLIPS     = 32
SMOKE_DURATION_S  = 60
SMOKE_SEED        = 0
SMOKE_SCENARIOS   = 1

# IMPORTANT: align soundscape labeling window with model input window
LABEL_WIN_S = float(CLIP_S)  # 5.0

class EvalConfig:
    def __init__(self, decision_hop_s=0.5, label_win_s=5.0, stage2_gate=0.30, stage2_uncert_lo=0.25, stage2_uncert_hi=0.65):
        self.decision_hop_s = decision_hop_s
        self.label_win_s = label_win_s
        self.stage2_gate = stage2_gate
        self.stage2_uncert_lo = stage2_uncert_lo
        self.stage2_uncert_hi = stage2_uncert_hi

EVAL_CFG_SMOKE = EvalConfig(
    decision_hop_s=0.5,
    label_win_s=LABEL_WIN_S,
    stage2_gate=0.30,
    stage2_uncert_lo=0.25,
    stage2_uncert_hi=0.65
)

# Stage1 gate: tight-ish but not too tight
gate_cfg_smoke = DSPGateConfig(
    sr=16000, frame_s=0.5, hop_s=0.05, n_fft=512,
    ema_alpha=0.02,
    delta_db=15.0,          # was 18 (too strict in 60s)
    flux_thr=0.40,          # was 0.45
    band_thr_db=14.0,
    cooldown_s=0.8,
    hysteresis_frames=3,
    sleep_db=-48.0,
    sleep_delta_db=12.0,
    sleep_flux_thr=0.35,
)
S1_SMOKE_GATE = DSPGate(gate_cfg_smoke, "B_energy_flux")
S1_SMOKE_NAME = "SMOKE_S1_B_energy_flux"

# -----------------------------
# Helpers: metrics (no sklearn)
# -----------------------------
def _prec_rec_f1(y_true, y_pred):
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    tp = int(((y_true == 1) & (y_pred == 1)).sum())
    fp = int(((y_true == 0) & (y_pred == 1)).sum())
    fn = int(((y_true == 1) & (y_pred == 0)).sum())
    prec = tp / (tp + fp + 1e-12)
    rec  = tp / (tp + fn + 1e-12)
    f1   = 2 * prec * rec / (prec + rec + 1e-12)
    return float(prec), float(rec), float(f1), tp, fp, fn

def _auroc_rank(y_true, y_score):
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    if len(np.unique(y_true)) < 2:
        return float("nan")
    pos = y_score[y_true==1]
    neg = y_score[y_true==0]
    if len(pos)==0 or len(neg)==0:
        return float("nan")
    cnt = 0.0
    for p in pos:
        cnt += np.sum(p > neg) + 0.5*np.sum(p == neg)
    return float(cnt / (len(pos)*len(neg)))

def _to_wave_tensor_from_filename(fn: str) -> torch.Tensor:
    p = resolve_audio_path(ESC50_32K_ROOT, fn)
    w = load_audio_32k_5s(p)  # (T,)
    assert w.shape == (CLIP_T,), f"bad wave shape: {w.shape}"
    return torch.from_numpy(w)[None, None, :].to(DEVICE)  # (1,1,T)

# -----------------------------
# Stage3 (single clips) smoke (no dependency on your earlier helper)
# -----------------------------
@torch.no_grad()
def stage3_single_clips_smoke(df_eval: pd.DataFrame, batch_size: int = 16):
    y_true = df_eval["is_emergency"].astype(int).values
    y_pred = []
    y_score = []
    y_t_true = df_eval["target"].astype(int).values
    y_t_pred = []

    em_targets = [cat_to_target[c] for c in EMERGENCY_CATS]

    for i in range(0, len(df_eval), batch_size):
        sub = df_eval.iloc[i:i+batch_size]
        waves = []
        for fn in sub["filename"].values:
            waves.append(load_audio_32k_5s(resolve_audio_path(ESC50_32K_ROOT, fn)))
        x = torch.from_numpy(np.stack(waves))[:, None, :].to(DEVICE)  # (B,1,T)

        xm = wave_to_mel_128(mel128, x)
        logits = dymn_50(xm)[0]               # (B,50)
        p = F.softmax(logits, dim=-1)         # (B,50)
        p_np = p.detach().cpu().numpy()

        pred_t = p_np.argmax(axis=-1)
        y_t_pred.append(pred_t)

        p_em = p_np[:, em_targets].sum(axis=-1)           # emergency probability mass
        y_score.append(p_em)
        y_pred.append((p_em >= 0.5).astype(int))

    y_pred = np.concatenate(y_pred)
    y_score = np.concatenate(y_score)
    y_t_pred = np.concatenate(y_t_pred)

    prec, rec, f1, tp, fp, fn = _prec_rec_f1(y_true, y_pred)
    auroc = _auroc_rank(y_true, y_score)
    top1 = float((y_t_pred == y_t_true).mean())

    duration_s = len(df_eval) * float(CLIP_S)
    cavg = compute_cavg_macs(duration_s, 0, 0, len(df_eval), MAC_STAGE1_PER_CALL, 0.0, MACS_STAGE3_DYMN)
    eui = f1 / max(1e-12, cavg)

    return {
        "pipeline": "S3_single_clip",
        "EF1": f1, "Prec": prec, "Rec": rec, "AUROC": auroc, "Top1_Acc": top1,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": 0.0, "Stage2_calls": 0.0, "Stage3_calls": float(len(df_eval)),
        "TP": tp, "FP": fp, "FN": fn
    }

# -----------------------------
# Stage2+Stage3 (single clips) smoke
# -----------------------------
@torch.no_grad()
def stage2_stage3_single_clips_smoke(df_eval: pd.DataFrame, stage2_name: str, stage2_type: str, cfg: EvalConfig,
                                     bc_frontend=None, bc_model=None):
    y_true, y_pred, y_score = [], [], []
    s2_calls, s3_calls = 0, 0

    for i in range(len(df_eval)):
        fn = df_eval.loc[i, "filename"]
        yt = int(df_eval.loc[i, "is_emergency"])
        x = _to_wave_tensor_from_filename(fn)

        if stage2_type == "mn":
            pdanger = float(stage2_mn_pdanger(x).item())
            mac_s2 = MACS_STAGE2_MN
        else:
            pdanger = float(bcresnet_predict_pdanger(bc_frontend, bc_model, x[:, 0, :]).item())
            mac_s2 = MACS_STAGE2_BC
        s2_calls += 1

        escalate = (pdanger >= cfg.stage2_gate)
        if cfg.stage2_uncert_lo is not None and cfg.stage2_uncert_hi is not None:
            escalate = escalate or (cfg.stage2_uncert_lo <= pdanger <= cfg.stage2_uncert_hi)

        if not escalate:
            yp = 0
        else:
            t = int(stage3_predict_target(x).item())
            cat = target_to_cat[t]
            yp = 1 if cat in EMERGENCY_CATS else 0
            s3_calls += 1

        y_true.append(yt); y_pred.append(yp); y_score.append(pdanger)

    prec, rec, f1, tp, fp, fn = _prec_rec_f1(y_true, y_pred)
    auroc = _auroc_rank(y_true, y_score)

    duration_s = len(df_eval) * float(CLIP_S)
    cavg = compute_cavg_macs(duration_s, 0, s2_calls, s3_calls, MAC_STAGE1_PER_CALL, mac_s2, MACS_STAGE3_DYMN)
    eui = f1 / max(1e-12, cavg)

    return {
        "pipeline": "S2+S3_single_clip",
        "stage2_name": stage2_name,
        "EF1": f1, "Prec": prec, "Rec": rec, "AUROC": auroc,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": 0.0, "Stage2_calls": float(s2_calls), "Stage3_calls": float(s3_calls),
        "TP": tp, "FP": fp, "FN": fn
    }

# -----------------------------
# Soundscape helpers: force >=1 emergency @ high SNR + align gating
# -----------------------------
def _rms(x: np.ndarray) -> float:
    return float(np.sqrt(np.mean(x.astype(np.float32)**2) + 1e-12))

def _mix_at_snr(bg: np.ndarray, sig: np.ndarray, snr_db: float) -> np.ndarray:
    r_bg = _rms(bg)
    r_sig = _rms(sig)
    target_r_sig = r_bg * (10.0 ** (snr_db / 20.0))
    scale = target_r_sig / (r_sig + 1e-12)
    return (sig * scale).astype(np.float32)

def ensure_min_emergencies_soundscape(wave: np.ndarray, ann: list, duration_s: float,
                                      min_emerg: int = 1, seed: int = 0, inject_snr_db: float = 12.0):
    rng = np.random.RandomState(seed)
    n_em = sum(1 for a in ann if a.get("kind") == "emergency")
    if n_em >= min_emerg:
        return wave, ann

    df_em = df[df["category"].isin(list(EMERGENCY_CATS))]
    assert len(df_em) > 0, "No emergency clips found in df; check EMERGENCY_CATS vs esc50.csv categories."
    row = df_em.sample(n=1, random_state=seed).iloc[0]
    clip = load_audio_32k_5s(resolve_audio_path(ESC50_32K_ROOT, row["filename"]))  # 5s

    # inject in middle
    t_s = max(CLIP_S, min(duration_s - CLIP_S, duration_s * 0.6))
    start = int(t_s * SR)
    end = min(len(wave), start + len(clip))
    clip = clip[:end - start]

    bg_seg = wave[start:end].copy()
    clip2 = _mix_at_snr(bg_seg, clip, inject_snr_db)

    wave2 = wave.copy()
    wave2[start:end] = np.clip(wave2[start:end] + clip2, -1.0, 1.0)

    a = {
        "t_start": start / SR,
        "t_end": end / SR,
        "category": str(row["category"]),
        "kind": "emergency",
        "snr_db": float(inject_snr_db),
        "forced": True,
    }
    ann2 = sorted(ann + [a], key=lambda d: d["t_start"])
    return wave2, ann2

def build_gt_emergency_windows_soundscape(ann, duration_s, decision_hop_s, win_s):
    # positive if overlaps trailing window [t-win_s, t]
    times = np.arange(decision_hop_s, duration_s + 1e-6, decision_hop_s)
    y = np.zeros(len(times), dtype=np.int64)
    for i, t in enumerate(times):
        w0, w1 = t - win_s, t
        for a in ann:
            if a.get("kind") != "emergency":
                continue
            if not (a["t_end"] <= w0 or a["t_start"] >= w1):
                y[i] = 1
                break
    return times, y

def get_trailing_window(wave, t_end_s, win_s):
    end = int(t_end_s * SR)
    win_T = int(win_s * SR)
    start = end - win_T
    if start < 0:
        seg = np.pad(wave[:end], (abs(start), 0))
    else:
        seg = wave[start:end]
    if len(seg) < win_T:
        seg = np.pad(seg, (0, win_T - len(seg)))
    return seg.astype(np.float32)

def stage1_trigger_times(stage1_gate, wave_32k):
    trig_hops, hop_s = stage1_gate.run(wave_32k)
    idx = np.where(trig_hops)[0]
    # end-of-frame time to reduce alignment misses
    frame_s = stage1_gate.cfg.frame_s
    trig_times = (idx * hop_s + frame_s).astype(np.float32)
    return trig_times, hop_s, len(trig_hops)

def gate_open_mask(decision_times, trig_times, hold_s):
    if trig_times is None or len(trig_times)==0:
        return np.zeros(len(decision_times), dtype=np.bool_)
    mask = np.zeros(len(decision_times), dtype=np.bool_)
    for i, t in enumerate(decision_times):
        w0, w1 = t - hold_s, t
        i0 = np.searchsorted(trig_times, w0, side="left")
        i1 = np.searchsorted(trig_times, w1, side="right")
        mask[i] = (i1 > i0)
    return mask

@torch.no_grad()
def run_soundscape_pipeline_smoke(wave, ann, pipeline, cfg: EvalConfig,
                                 stage1_gate=None,
                                 stage2_type=None, stage2_name=None,
                                 bc_frontend=None, bc_model=None):
    duration_s = len(wave) / SR
    decision_times, y_true = build_gt_emergency_windows_soundscape(
        ann, duration_s, cfg.decision_hop_s, cfg.label_win_s
    )

    # stage1 gating
    stage1_hops = 0
    stage1_events = 0
    open_mask = None
    if stage1_gate is not None:
        trig_times, s1_hop, stage1_hops = stage1_trigger_times(stage1_gate, wave)
        stage1_events = int(len(trig_times))
        open_mask = gate_open_mask(decision_times, trig_times, hold_s=cfg.label_win_s)

    y_pred = np.zeros(len(decision_times), dtype=np.int64)
    y_score = np.zeros(len(decision_times), dtype=np.float32)
    n_s2 = 0
    n_s3 = 0

    em_targets = [cat_to_target[c] for c in EMERGENCY_CATS]

    for i, t in enumerate(decision_times):
        run_s2 = False
        run_s3 = False

        if pipeline == "S3":
            run_s3 = True
        elif pipeline == "S1+S3":
            run_s3 = bool(open_mask[i])
        elif pipeline == "S2+S3":
            run_s2 = True
        elif pipeline == "S1+S2+S3":
            run_s2 = bool(open_mask[i])
        else:
            raise ValueError(pipeline)

        seg = get_trailing_window(wave, float(t), cfg.label_win_s)
        x = torch.from_numpy(seg)[None, None, :].to(DEVICE)

        if run_s2:
            n_s2 += 1
            if stage2_type == "mn":
                pd = float(stage2_mn_pdanger(x).item())
                mac_s2 = MACS_STAGE2_MN
            else:
                pd = float(bcresnet_predict_pdanger(bc_frontend, bc_model, x[:,0,:]).item())
                mac_s2 = MACS_STAGE2_BC
            y_score[i] = pd

            escalate = (pd >= cfg.stage2_gate)
            if cfg.stage2_uncert_lo is not None and cfg.stage2_uncert_hi is not None:
                escalate = escalate or (cfg.stage2_uncert_lo <= pd <= cfg.stage2_uncert_hi)

            run_s3 = escalate
        else:
            mac_s2 = 0.0

        if run_s3:
            n_s3 += 1
            # predict class
            t_hat = int(stage3_predict_target(x).item())
            cat = target_to_cat[t_hat]
            y_pred[i] = 1 if cat in EMERGENCY_CATS else 0

            # if no stage2 score, use stage3 emergency prob mass
            if not run_s2:
                xm = wave_to_mel_128(mel128, x)
                logits = dymn_50(xm)[0]  # (1,50)
                p = F.softmax(logits, dim=-1)[0].detach().cpu().numpy()
                y_score[i] = float(p[em_targets].sum())

    # metrics
    prec, rec, f1, tp, fp, fn = _prec_rec_f1(y_true, y_pred)
    auroc = _auroc_rank(y_true, y_score)

    # TR/DER
    tr = float(stage1_events / max(1, stage1_hops)) if stage1_gate is not None else float("nan")
    if stage1_gate is not None:
        der = float(n_s3 / max(1, stage1_events))
    else:
        der = float(n_s3 / max(1, len(decision_times)))

    # compute
    if stage2_type == "mn":
        mac_s2_call = MACS_STAGE2_MN
    elif stage2_type == "bc":
        mac_s2_call = MACS_STAGE2_BC
    else:
        mac_s2_call = 0.0

    cavg = compute_cavg_macs(
        duration_s=duration_s,
        n_stage1_calls=(stage1_hops if stage1_gate is not None else 0),
        n_stage2_calls=n_s2,
        n_stage3_calls=n_s3,
        mac_stage1_per_call=MAC_STAGE1_PER_CALL,
        mac_stage2_per_call=mac_s2_call,
        mac_stage3_per_call=MACS_STAGE3_DYMN,
    )
    eui = f1 / max(1e-12, cavg)

    return {
        "pipeline": pipeline,
        "EF1": f1, "Prec": prec, "Rec": rec, "AUROC": auroc,
        "TR": tr, "DER": der,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": float(stage1_hops if stage1_gate is not None else 0),
        "Stage2_calls": float(n_s2),
        "Stage3_calls": float(n_s3),
        "TP": tp, "FP": fp, "FN": fn,
        "stage2_name": stage2_name or "",
        "Stage1": (S1_SMOKE_NAME if stage1_gate is not None else "")
    }

# ============================================================
# Build smoke eval df (balanced)
# ============================================================
df_pos = df[df["is_emergency"] == 1]
df_neg = df[df["is_emergency"] == 0]
n_pos = min(16, len(df_pos))
n_neg = min(16, len(df_neg))
assert n_pos > 0 and n_neg > 0, "Need both emergency and non-emergency clips for smoke."

df_eval_smoke = pd.concat(
    [df_pos.sample(n=n_pos, random_state=SMOKE_SEED),
     df_neg.sample(n=n_neg, random_state=SMOKE_SEED)],
    ignore_index=True
).sample(frac=1.0, random_state=SMOKE_SEED).reset_index(drop=True).iloc[:SMOKE_N_CLIPS].copy()

print(f"[SMOKE] df_eval_smoke: {len(df_eval_smoke)} | emergency: {int(df_eval_smoke['is_emergency'].sum())} | non-em: {int((df_eval_smoke['is_emergency']==0).sum())}")

# ============================================================
# Pick 1 scenario @ 60s
# ============================================================
SCENARIOS_SMOKE = []
for s in SCENARIOS[:SMOKE_SCENARIOS]:
    SCENARIOS_SMOKE.append(ScenarioConfig(**{**s.__dict__, "duration_s": SMOKE_DURATION_S}))
scen = SCENARIOS_SMOKE[0]
print("[SMOKE] Scenario:", scen.name, scen.duration_s, "seconds")

# ============================================================
# Load BC tau=8
# ============================================================
bc_name = "bcresnet_tau8"
assert bc_name in BCRESNET_CKPTS, f"BCRESNET_CKPTS missing key: {bc_name}"
tau8, fe8, m8 = load_bcresnet_binary(BCRESNET_CKPTS[bc_name], device=DEVICE)
print("[SMOKE] Loaded", bc_name, "tau =", tau8)

# Quick forward sanity
x0 = _to_wave_tensor_from_filename(df_eval_smoke.loc[0, "filename"])
p_mn = float(stage2_mn_pdanger(x0).item())
p_bc = float(bcresnet_predict_pdanger(fe8, m8, x0[:,0,:]).item())
t0 = int(stage3_predict_target(x0).item())
print("[SMOKE] MN p(danger):", p_mn)
print("[SMOKE] BC p(danger):", p_bc)
print("[SMOKE] DyMN pred target:", t0, "| cat:", target_to_cat[t0])

# ============================================================
# RUN SMOKE EXPERIMENTS
# ============================================================
smoke_results = []

# (1) Stage3 single clips
r = stage3_single_clips_smoke(df_eval_smoke, batch_size=16)
r.update({"Experiment": "SMOKE_Stage3_single_clips", "Scenario": "", "stage2_name": ""})
smoke_results.append(r)
print("[SMOKE] OK Stage3 single clips | EF1:", r["EF1"], "| Cavg:", r["Cavg_MACs_per_s"])

# (2) Stage2+Stage3 single clips (MN)
r = stage2_stage3_single_clips_smoke(df_eval_smoke, "mn04_binary", "mn", EVAL_CFG_SMOKE)
r.update({"Experiment": "SMOKE_Stage2+Stage3_single_clips", "Scenario": ""})
smoke_results.append(r)
print("[SMOKE] OK MN S2+S3 clips | EF1:", r["EF1"], "| S3 calls:", r["Stage3_calls"], "| Cavg:", r["Cavg_MACs_per_s"])

# (2b) Stage2+Stage3 single clips (BC tau8)
r = stage2_stage3_single_clips_smoke(df_eval_smoke, bc_name, "bc", EVAL_CFG_SMOKE, bc_frontend=fe8, bc_model=m8)
r.update({"Experiment": "SMOKE_Stage2+Stage3_single_clips", "Scenario": ""})
smoke_results.append(r)
print("[SMOKE] OK BC S2+S3 clips | EF1:", r["EF1"], "| S3 calls:", r["Stage3_calls"], "| Cavg:", r["Cavg_MACs_per_s"])

# (3) Build 60s soundscape + force >=1 emergency (high SNR so Stage1 sees it)
wave_sc, ann_sc = build_soundscape(scen, seed=2026)
wave_sc, ann_sc = ensure_min_emergencies_soundscape(wave_sc, ann_sc, scen.duration_s, min_emerg=1, seed=999, inject_snr_db=12.0)

print("[SMOKE] Soundscape:", scen.name,
      "len(s)=", len(wave_sc)/SR,
      "events=", len(ann_sc),
      "emerg=", sum(1 for a in ann_sc if a.get("kind")=="emergency"))

# Optional: show emergency interval(s)
em = [a for a in ann_sc if a.get("kind")=="emergency"]
print("[SMOKE] Emergency intervals:", [(a["t_start"], a["t_end"], a.get("category"), a.get("snr_db"), a.get("forced")) for a in em])

PIPELINES = [
    ("S3",       None,                 None),
    ("S1+S3",    S1_SMOKE_GATE,         None),
    ("S2+S3",    None,                 ("mn04_binary", "mn")),
    ("S1+S2+S3", S1_SMOKE_GATE,        ("mn04_binary", "mn")),
]

for pipe, gate, s2 in PIPELINES:
    if s2 is None:
        rr = run_soundscape_pipeline_smoke(
            wave_sc, ann_sc,
            pipeline=pipe,
            cfg=EVAL_CFG_SMOKE,
            stage1_gate=gate,
            stage2_type=None,
            stage2_name=None
        )
    else:
        s2_name, s2_type = s2
        rr = run_soundscape_pipeline_smoke(
            wave_sc, ann_sc,
            pipeline=pipe,
            cfg=EVAL_CFG_SMOKE,
            stage1_gate=gate,
            stage2_type=s2_type,
            stage2_name=s2_name
        )

    rr.update({"Experiment": f"SMOKE_{pipe}", "Scenario": scen.name})
    smoke_results.append(rr)

    print(f"[SMOKE] OK {pipe:8s} | EF1={rr['EF1']:.3f} AUROC={rr['AUROC']:.3f} "
          f"TR={rr['TR'] if not np.isnan(rr['TR']) else float('nan'):.4f} DER={rr['DER']:.4f} "
          f"Cavg={rr['Cavg_MACs_per_s']:.2e} | calls S1/S2/S3 = {rr['Stage1_calls']:.0f}/{rr['Stage2_calls']:.0f}/{rr['Stage3_calls']:.0f}")

# ============================================================
# SAVE + SHOW TABLE
# ============================================================
smoke_df = pd.DataFrame(smoke_results)

cols = [
    "Experiment", "Scenario", "Stage1", "stage2_name", "pipeline",
    "EF1", "Prec", "Rec", "AUROC",
    "TR", "DER",
    "Cavg_MACs_per_s", "EUI",
    "Stage1_calls", "Stage2_calls", "Stage3_calls",
    "Top1_Acc",
    "TP", "FP", "FN"
]
for c in cols:
    if c not in smoke_df.columns:
        smoke_df[c] = np.nan
smoke_df = smoke_df[cols]

os.makedirs(OUT_DIR, exist_ok=True)
smoke_path = os.path.join(OUT_DIR, "smoke_results.csv")
smoke_df.to_csv(smoke_path, index=False)
print("[SMOKE] Saved:", smoke_path)

smoke_df


[SMOKE] df_eval_smoke: 32 | emergency: 16 | non-em: 16
[SMOKE] Scenario: quiet_home_day 60 seconds
[SMOKE] Loaded bcresnet_tau8 tau = 8.0
[SMOKE] MN p(danger): 0.8393261432647705
[SMOKE] BC p(danger): 0.8105898499488831
[SMOKE] DyMN pred target: 19 | cat: thunderstorm
[SMOKE] OK Stage3 single clips | EF1: 0.9677419354833091 | Cavg: 11607404.8
[SMOKE] OK MN S2+S3 clips | EF1: 0.9677419354833091 | S3 calls: 16.0 | Cavg: 18889833.6
[SMOKE] OK BC S2+S3 clips | EF1: 0.9677419354833091 | S3 calls: 22.0 | Cavg: 11584498.0
[SMOKE] Soundscape: quiet_home_day len(s)= 60.0 events= 3 emerg= 1
[SMOKE] Emergency intervals: [(36.0, 41.0, 'dog', 12.0, True)]
[SMOKE] OK S3       | EF1=0.690 AUROC=0.593 TR=nan DER=1.0000 Cavg=1.16e+08 | calls S1/S2/S3 = 0/0/120
[SMOKE] OK S1+S3    | EF1=0.690 AUROC=0.763 TR=0.0008 DER=10.0000 Cavg=1.04e+07 | calls S1/S2/S3 = 1191/0/10
[SMOKE] OK S2+S3    | EF1=0.593 AUROC=0.705 TR=nan DER=0.0667 Cavg=1.39e+08 | calls S1/S2/S3 = 0/120/8
[SMOKE] OK S1+S2+S3 | EF1=0.593 AU

,Experiment,Scenario,Stage1,stage2_name,pipeline,EF1,Prec,Rec,AUROC,TR,DER,Cavg_MACs_per_s,EUI,Stage1_calls,Stage2_calls,Stage3_calls,Top1_Acc,TP,FP,FN
0,SMOKE_Stage3_single_clips,,NaN,,S3_single_clip,0.967742,1.0,0.937500,1.000000,NaN,NaN,1.160740e+07,8.337281e-08,0.0,0.0,32.0,1.0,15,0,1
1,SMOKE_Stage2+Stage3_single_clips,,NaN,mn04_binary,S2+S3_single_clip,0.967742,1.0,0.937500,0.996094,NaN,NaN,1.888983e+07,5.123083e-08,0.0,32.0,16.0,NaN,15,0,1
2,SMOKE_Stage2+Stage3_single_clips,,NaN,bcresnet_tau8,S2+S3_single_clip,0.967742,1.0,0.937500,0.910156,NaN,NaN,1.158450e+07,8.353767e-08,0.0,32.0,22.0,NaN,15,0,1
3,SMOKE_S3,quiet_home_day,,,S3,0.689655,1.0,0.526316,0.593017,NaN,1.000000,1.160740e+08,5.941510e-09,0.0,0.0,120.0,NaN,10,0,9
4,SMOKE_S1+S3,quiet_home_day,SMOKE_S1_B_energy_flux,,S1+S3,0.689655,1.0,0.526316,0.763158,0.00084,10.000000,1.040459e+07,6.628376e-08,1191.0,0.0,10.0,NaN,10,0,9
5,SMOKE_S2+S3,quiet_home_day,,mn04_binary,S2+S3,0.592593,1.0,0.421053,0.705055,NaN,0.066667,1.385996e+08,4.275573e-09,0.0,120.0,8.0,NaN,8,0,11
6,SMOKE_S1+S2+S3,quiet_home_day,SMOKE_S1_B_energy_flux,mn04_binary,S1+S2+S3,0.592593,1.0,0.421053,0.763158,0.00084,8.000000,1.937513e+07,3.058522e-08,1191.0,10.0,8.0,NaN,8,0,11


In [ ]:
# ============================================================
# BIG RUN
# - Uses the SAME semantics as the smoke run:
#   * Soundscape GT positives: trailing window [t-LABEL_WIN_S, t] overlaps any emergency
#   * Model input on soundscapes: trailing window of length LABEL_WIN_S (== CLIP_S)
#   * Stage1 trigger time = end-of-frame; gate hold = LABEL_WIN_S
#   * No recursion; no reliance on old run_single_clip_stage2_stage3
# - Experiments:
#   1) Stage3 (single ESC-50 clips)
#   2) Stage2 + Stage3 (single ESC-50 clips) for MN + all BC taus
#   3) Soundscapes: S3, S1+S3, S2+S3, S1+S2+S3 across scenarios, Stage1 variants, Stage2 variants
# - Output: OUT_DIR/results_summary.csv
# ============================================================

import os, warnings, math
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

# -----------------------------
# (optional) silence EfficientAT warning
# -----------------------------
warnings.filterwarnings("ignore", category=FutureWarning, message=".*autocast.*deprecated.*")

# -----------------------------
# Safety checks (must exist from earlier cells)
# -----------------------------
needed = [
    "df", "ESC50_32K_ROOT", "resolve_audio_path", "load_audio_32k_5s", "SR", "CLIP_T", "CLIP_S",
    "EMERGENCY_CATS", "target_to_cat", "cat_to_target",
    "SCENARIOS", "ScenarioConfig", "build_soundscape",
    "BCRESNET_CKPTS", "load_bcresnet_binary", "bcresnet_predict_pdanger",
    "stage2_mn_pdanger", "stage3_predict_target",
    "DSP_GATES",  # dict: {name: DSPGate}
    "OUT_DIR", "DEVICE",
    "MAC_STAGE1_PER_CALL", "MACS_STAGE2_MN", "MACS_STAGE2_BC", "MACS_STAGE3_DYMN", "compute_cavg_macs",
    "mel128", "wave_to_mel_128", "dymn_50"
]
missing = [x for x in needed if x not in globals()]
assert not missing, f"Missing definitions: {missing}"

# ============================================================
# Config (match smoke semantics)
# ============================================================
class EvalConfig:
    def __init__(self, decision_hop_s=0.5, label_win_s=5.0, stage2_gate=0.30, stage2_uncert_lo=0.25, stage2_uncert_hi=0.65):
        self.decision_hop_s = decision_hop_s
        self.label_win_s = label_win_s
        self.stage2_gate = stage2_gate
        self.stage2_uncert_lo = stage2_uncert_lo
        self.stage2_uncert_hi = stage2_uncert_hi

EVAL_CFG = EvalConfig(
    decision_hop_s=0.5,
    label_win_s=float(CLIP_S),  # critical alignment: window = model input = 5s
    stage2_gate=0.30,
    stage2_uncert_lo=0.25,
    stage2_uncert_hi=0.65
)

BATCH_SIZE = 64
SOUNDSCAPE_SEED_BASE = 2026

# ============================================================
# Helpers: metrics + IO
# ============================================================
def _prec_rec_f1(y_true, y_pred):
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    tp = int(((y_true == 1) & (y_pred == 1)).sum())
    fp = int(((y_true == 0) & (y_pred == 1)).sum())
    fn = int(((y_true == 1) & (y_pred == 0)).sum())
    prec = tp / (tp + fp + 1e-12)
    rec  = tp / (tp + fn + 1e-12)
    f1   = 2 * prec * rec / (prec + rec + 1e-12)
    return float(prec), float(rec), float(f1), tp, fp, fn

def _auroc_rank(y_true, y_score):
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    if len(np.unique(y_true)) < 2:
        return float("nan")
    pos = y_score[y_true==1]
    neg = y_score[y_true==0]
    if len(pos)==0 or len(neg)==0:
        return float("nan")
    cnt = 0.0
    for p in pos:
        cnt += np.sum(p > neg) + 0.5*np.sum(p == neg)
    return float(cnt / (len(pos)*len(neg)))

def _to_wave_tensor_from_filename(fn: str) -> torch.Tensor:
    p = resolve_audio_path(ESC50_32K_ROOT, fn)
    w = load_audio_32k_5s(p)
    assert w.shape == (CLIP_T,), f"bad wave shape: {w.shape}"
    return torch.from_numpy(w)[None, None, :].to(DEVICE)

def build_gt_emergency_windows_soundscape(ann, duration_s, decision_hop_s, win_s):
    # positive if overlaps trailing window [t-win_s, t]
    times = np.arange(decision_hop_s, duration_s + 1e-6, decision_hop_s)
    y = np.zeros(len(times), dtype=np.int64)
    for i, t in enumerate(times):
        w0, w1 = t - win_s, t
        for a in ann:
            if a.get("kind") != "emergency":
                continue
            if not (a["t_end"] <= w0 or a["t_start"] >= w1):
                y[i] = 1
                break
    return times, y

def get_trailing_window(wave, t_end_s, win_s):
    end = int(t_end_s * SR)
    win_T = int(win_s * SR)
    start = end - win_T
    if start < 0:
        seg = np.pad(wave[:end], (abs(start), 0))
    else:
        seg = wave[start:end]
    if len(seg) < win_T:
        seg = np.pad(seg, (0, win_T - len(seg)))
    return seg.astype(np.float32)

def stage1_trigger_times(stage1_gate, wave_32k):
    trig_hops, hop_s = stage1_gate.run(wave_32k)
    idx = np.where(trig_hops)[0]
    frame_s = stage1_gate.cfg.frame_s
    trig_times = (idx * hop_s + frame_s).astype(np.float32)  # end-of-frame
    return trig_times, hop_s, len(trig_hops), int(len(trig_times))

def gate_open_mask(decision_times, trig_times, hold_s):
    if trig_times is None or len(trig_times)==0:
        return np.zeros(len(decision_times), dtype=np.bool_)
    mask = np.zeros(len(decision_times), dtype=np.bool_)
    for i, t in enumerate(decision_times):
        w0, w1 = t - hold_s, t
        i0 = np.searchsorted(trig_times, w0, side="left")
        i1 = np.searchsorted(trig_times, w1, side="right")
        mask[i] = (i1 > i0)
    return mask

# (Optional) time-to-detect: first decision time where pred=1 after the first emergency start
def _ttd_seconds(ann, decision_times, y_pred):
    em = [a for a in ann if a.get("kind")=="emergency"]
    if len(em)==0:
        return float("nan")
    t0 = float(min(a["t_start"] for a in em))
    for t, yp in zip(decision_times, y_pred):
        if t >= t0 and int(yp)==1:
            return float(t - t0)
    return float("nan")

# ============================================================
# 1) Single-clip evaluators (Stage3, Stage2+Stage3)
# ============================================================
@torch.no_grad()
def eval_stage3_single_clips(df_eval: pd.DataFrame, batch_size: int = 64):
    y_true = df_eval["is_emergency"].astype(int).values
    y_t_true = df_eval["target"].astype(int).values
    y_pred = []
    y_score = []
    y_t_pred = []

    em_targets = [cat_to_target[c] for c in EMERGENCY_CATS]

    for i in range(0, len(df_eval), batch_size):
        sub = df_eval.iloc[i:i+batch_size]
        waves = []
        for fn in sub["filename"].values:
            waves.append(load_audio_32k_5s(resolve_audio_path(ESC50_32K_ROOT, fn)))
        x = torch.from_numpy(np.stack(waves))[:, None, :].to(DEVICE)  # (B,1,T)

        xm = wave_to_mel_128(mel128, x)
        logits = dymn_50(xm)[0]               # (B,50)
        p = F.softmax(logits, dim=-1)         # (B,50)
        p_np = p.detach().cpu().numpy()

        pred_t = p_np.argmax(axis=-1)
        y_t_pred.append(pred_t)

        p_em = p_np[:, em_targets].sum(axis=-1)
        y_score.append(p_em)
        y_pred.append((p_em >= 0.5).astype(int))

    y_pred = np.concatenate(y_pred)
    y_score = np.concatenate(y_score)
    y_t_pred = np.concatenate(y_t_pred)

    prec, rec, f1, tp, fp, fn = _prec_rec_f1(y_true, y_pred)
    auroc = _auroc_rank(y_true, y_score)
    top1 = float((y_t_pred == y_t_true).mean())

    duration_s = len(df_eval) * float(CLIP_S)
    cavg = compute_cavg_macs(duration_s, 0, 0, len(df_eval), MAC_STAGE1_PER_CALL, 0.0, MACS_STAGE3_DYMN)
    eui = f1 / max(1e-12, cavg)

    return {
        "pipeline": "S3_single_clip",
        "stage2_name": "",
        "EF1": f1, "Prec": prec, "Rec": rec, "AUROC": auroc, "Top1_Acc": top1,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": 0.0, "Stage2_calls": 0.0, "Stage3_calls": float(len(df_eval)),
        "TP": tp, "FP": fp, "FN": fn
    }

@torch.no_grad()
def eval_stage2_stage3_single_clips(df_eval: pd.DataFrame, stage2_name: str, stage2_type: str, cfg: EvalConfig,
                                    bc_frontend=None, bc_model=None):
    y_true, y_pred, y_score = [], [], []
    s2_calls, s3_calls = 0, 0

    for i in range(len(df_eval)):
        fn = df_eval.loc[i, "filename"]
        yt = int(df_eval.loc[i, "is_emergency"])
        x = _to_wave_tensor_from_filename(fn)

        if stage2_type == "mn":
            pd = float(stage2_mn_pdanger(x).item())
            mac_s2 = MACS_STAGE2_MN
        else:
            pd = float(bcresnet_predict_pdanger(bc_frontend, bc_model, x[:, 0, :]).item())
            mac_s2 = MACS_STAGE2_BC
        s2_calls += 1

        escalate = (pd >= cfg.stage2_gate)
        if cfg.stage2_uncert_lo is not None and cfg.stage2_uncert_hi is not None:
            escalate = escalate or (cfg.stage2_uncert_lo <= pd <= cfg.stage2_uncert_hi)

        if not escalate:
            yp = 0
        else:
            t = int(stage3_predict_target(x).item())
            cat = target_to_cat[t]
            yp = 1 if cat in EMERGENCY_CATS else 0
            s3_calls += 1

        y_true.append(yt); y_pred.append(yp); y_score.append(pd)

    prec, rec, f1, tp, fp, fn = _prec_rec_f1(y_true, y_pred)
    auroc = _auroc_rank(y_true, y_score)

    duration_s = len(df_eval) * float(CLIP_S)
    cavg = compute_cavg_macs(duration_s, 0, s2_calls, s3_calls, MAC_STAGE1_PER_CALL, mac_s2, MACS_STAGE3_DYMN)
    eui = f1 / max(1e-12, cavg)

    return {
        "pipeline": "S2+S3_single_clip",
        "stage2_name": stage2_name,
        "EF1": f1, "Prec": prec, "Rec": rec, "AUROC": auroc, "Top1_Acc": np.nan,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": 0.0, "Stage2_calls": float(s2_calls), "Stage3_calls": float(s3_calls),
        "TP": tp, "FP": fp, "FN": fn
    }

# ============================================================
# 2) Soundscape evaluator (S3, S1+S3, S2+S3, S1+S2+S3)
# ============================================================
@torch.no_grad()
def eval_soundscape_pipeline(wave, ann, pipeline, cfg: EvalConfig,
                             stage1_gate=None,
                             stage2_type=None, stage2_name=None,
                             bc_frontend=None, bc_model=None):
    duration_s = len(wave) / SR
    decision_times, y_true = build_gt_emergency_windows_soundscape(ann, duration_s, cfg.decision_hop_s, cfg.label_win_s)

    stage1_hops = 0
    stage1_events = 0
    open_mask = None
    if stage1_gate is not None:
        trig_times, hop_s, stage1_hops, stage1_events = stage1_trigger_times(stage1_gate, wave)
        open_mask = gate_open_mask(decision_times, trig_times, hold_s=cfg.label_win_s)

    y_pred = np.zeros(len(decision_times), dtype=np.int64)
    y_score = np.zeros(len(decision_times), dtype=np.float32)
    n_s2 = 0
    n_s3 = 0

    em_targets = [cat_to_target[c] for c in EMERGENCY_CATS]

    for i, t in enumerate(decision_times):
        run_s2 = False
        run_s3 = False

        if pipeline == "S3":
            run_s3 = True
        elif pipeline == "S1+S3":
            run_s3 = bool(open_mask[i])
        elif pipeline == "S2+S3":
            run_s2 = True
        elif pipeline == "S1+S2+S3":
            run_s2 = bool(open_mask[i])
        else:
            raise ValueError(pipeline)

        seg = get_trailing_window(wave, float(t), cfg.label_win_s)
        x = torch.from_numpy(seg)[None, None, :].to(DEVICE)

        if run_s2:
            n_s2 += 1
            if stage2_type == "mn":
                pd = float(stage2_mn_pdanger(x).item())
                mac_s2_call = MACS_STAGE2_MN
            else:
                pd = float(bcresnet_predict_pdanger(bc_frontend, bc_model, x[:,0,:]).item())
                mac_s2_call = MACS_STAGE2_BC
            y_score[i] = pd

            escalate = (pd >= cfg.stage2_gate)
            if cfg.stage2_uncert_lo is not None and cfg.stage2_uncert_hi is not None:
                escalate = escalate or (cfg.stage2_uncert_lo <= pd <= cfg.stage2_uncert_hi)

            run_s3 = escalate
        else:
            mac_s2_call = 0.0

        if run_s3:
            n_s3 += 1
            t_hat = int(stage3_predict_target(x).item())
            cat = target_to_cat[t_hat]
            y_pred[i] = 1 if cat in EMERGENCY_CATS else 0

            # if no Stage2 score, use Stage3 emergency prob mass as score
            if not run_s2:
                xm = wave_to_mel_128(mel128, x)
                logits = dymn_50(xm)[0]  # (1,50)
                p = F.softmax(logits, dim=-1)[0].detach().cpu().numpy()
                y_score[i] = float(p[em_targets].sum())

    prec, rec, f1, tp, fp, fn = _prec_rec_f1(y_true, y_pred)
    auroc = _auroc_rank(y_true, y_score)
    ttd = _ttd_seconds(ann, decision_times, y_pred)

    # CLEAN + interpretable gate metrics:
    #   TR_s1_per_min: Stage1 trigger events per minute (only for Stage1 pipelines)
    #   S3_rate: fraction of decision steps where Stage3 ran (directly interpretable)
    tr_per_min = float(stage1_events / max(1e-9, duration_s/60.0)) if stage1_gate is not None else float("nan")
    s3_rate = float(n_s3 / max(1, len(decision_times)))

    # compute
    if stage2_type == "mn":
        mac_s2_per_call = MACS_STAGE2_MN
    elif stage2_type == "bc":
        mac_s2_per_call = MACS_STAGE2_BC
    else:
        mac_s2_per_call = 0.0

    cavg = compute_cavg_macs(
        duration_s=duration_s,
        n_stage1_calls=(stage1_hops if stage1_gate is not None else 0),
        n_stage2_calls=n_s2,
        n_stage3_calls=n_s3,
        mac_stage1_per_call=MAC_STAGE1_PER_CALL,
        mac_stage2_per_call=mac_s2_per_call,
        mac_stage3_per_call=MACS_STAGE3_DYMN,
    )
    eui = f1 / max(1e-12, cavg)

    return {
        "pipeline": pipeline,
        "stage2_name": stage2_name or "",
        "EF1": f1, "Prec": prec, "Rec": rec, "AUROC": auroc,
        "TR": tr_per_min,           # repurposed: Stage1 triggers per minute
        "DER": s3_rate,             # repurposed: Stage3 duty rate on decision grid
        "TTD_sec": ttd,
        "Cavg_MACs_per_s": cavg, "EUI": eui,
        "Stage1_calls": float(stage1_hops if stage1_gate is not None else 0),
        "Stage2_calls": float(n_s2),
        "Stage3_calls": float(n_s3),
        "Top1_Acc": np.nan,
        "TP": tp, "FP": fp, "FN": fn
    }

# ============================================================
# Prepare eval split (all 2000 clips)
# ============================================================
df_eval = df.copy().reset_index(drop=True)
print("[BIG] df_eval:", len(df_eval), "| emergencies:", int(df_eval["is_emergency"].sum()))

# ============================================================
# Load all Stage2 variants (MN + all BC ckpts)
# ============================================================
BC_MODELS = {}
for name, ckpt in BCRESNET_CKPTS.items():
    tau, fe, m = load_bcresnet_binary(ckpt, device=DEVICE)
    BC_MODELS[name] = {"tau": tau, "frontend": fe, "model": m}
print("[BIG] Loaded BCResNet variants:", list(BC_MODELS.keys()))

STAGE2_LIST = [("mn04_binary", "mn", None)]
for name in BC_MODELS.keys():
    STAGE2_LIST.append((name, "bc", BC_MODELS[name]))
print("[BIG] Stage2 models:", [x[0] for x in STAGE2_LIST])

# ============================================================
# Run experiments
# ============================================================
results = []

# -------------------------
# 1) Single clips
# -------------------------
r = eval_stage3_single_clips(df_eval, batch_size=BATCH_SIZE)
r.update({"Experiment": "Stage3_single_clips", "Scenario": "", "Stage1": ""})
results.append(r)
print("[BIG] OK Stage3_single_clips | EF1:", r["EF1"], "| Cavg:", r["Cavg_MACs_per_s"])

for s2_name, s2_type, bc_pack in STAGE2_LIST:
    if s2_type == "mn":
        r = eval_stage2_stage3_single_clips(df_eval, s2_name, "mn", EVAL_CFG)
    else:
        r = eval_stage2_stage3_single_clips(df_eval, s2_name, "bc", EVAL_CFG,
                                            bc_frontend=bc_pack["frontend"], bc_model=bc_pack["model"])
    r.update({"Experiment": "Stage2+Stage3_single_clips", "Scenario": "", "Stage1": ""})
    results.append(r)
    print("[BIG] OK S2+S3 clips:", s2_name, "| EF1:", r["EF1"], "| S3 calls:", r["Stage3_calls"], "| Cavg:", r["Cavg_MACs_per_s"])

# -------------------------
# 2) Soundscapes
# -------------------------
for scen_idx, scen in enumerate(SCENARIOS):
    seed = SOUNDSCAPE_SEED_BASE + scen_idx*97
    wave, ann = build_soundscape(scen, seed=seed)

    # 2a) S3 only
    r = eval_soundscape_pipeline(wave, ann, pipeline="S3", cfg=EVAL_CFG,
                                 stage1_gate=None, stage2_type=None, stage2_name=None)
    r.update({"Experiment": "Stage3_soundscapes", "Scenario": scen.name, "Stage1": ""})
    results.append(r)
    print("[BIG] OK S3 soundscape:", scen.name, "| EF1:", r["EF1"], "| S3 rate:", r["DER"])

    # 2b) S1 + S3 (all DSP gates)
    for s1_name, gate in DSP_GATES.items():
        r = eval_soundscape_pipeline(wave, ann, pipeline="S1+S3", cfg=EVAL_CFG,
                                     stage1_gate=gate, stage2_type=None, stage2_name=None)
        r.update({"Experiment": "Stage1+Stage3_soundscapes", "Scenario": scen.name, "Stage1": s1_name})
        results.append(r)

    # 2c) S2 + S3 (all Stage2)
    for s2_name, s2_type, bc_pack in STAGE2_LIST:
        if s2_type == "mn":
            r = eval_soundscape_pipeline(wave, ann, pipeline="S2+S3", cfg=EVAL_CFG,
                                         stage1_gate=None, stage2_type="mn", stage2_name=s2_name)
        else:
            r = eval_soundscape_pipeline(wave, ann, pipeline="S2+S3", cfg=EVAL_CFG,
                                         stage1_gate=None, stage2_type="bc", stage2_name=s2_name,
                                         bc_frontend=bc_pack["frontend"], bc_model=bc_pack["model"])
        r.update({"Experiment": "Stage2+Stage3_soundscapes", "Scenario": scen.name, "Stage1": ""})
        results.append(r)

    # 2d) S1 + S2 + S3 (DSP × Stage2)
    for s1_name, gate in DSP_GATES.items():
        for s2_name, s2_type, bc_pack in STAGE2_LIST:
            if s2_type == "mn":
                r = eval_soundscape_pipeline(wave, ann, pipeline="S1+S2+S3", cfg=EVAL_CFG,
                                             stage1_gate=gate, stage2_type="mn", stage2_name=s2_name)
            else:
                r = eval_soundscape_pipeline(wave, ann, pipeline="S1+S2+S3", cfg=EVAL_CFG,
                                             stage1_gate=gate, stage2_type="bc", stage2_name=s2_name,
                                             bc_frontend=bc_pack["frontend"], bc_model=bc_pack["model"])
            r.update({"Experiment": "Stage1+Stage2+Stage3_soundscapes", "Scenario": scen.name, "Stage1": s1_name})
            results.append(r)

print("[BIG] Done. Total rows:", len(results))

# ============================================================
# Summarize (neat report columns only; no redundancy)
# ============================================================
res_df = pd.DataFrame(results)

keep_cols = [
    "Experiment", "Scenario", "Stage1", "stage2_name", "pipeline",
    "EF1", "Prec", "Rec", "AUROC",
    "TR", "DER", "TTD_sec",                    # TR=Stage1 triggers/min, DER=Stage3 duty rate
    "Cavg_MACs_per_s", "EUI",
    "Stage1_calls", "Stage2_calls", "Stage3_calls",
    "Top1_Acc"
]
for c in keep_cols:
    if c not in res_df.columns:
        res_df[c] = np.nan
res_df = res_df[keep_cols].copy()

# Sort: best EUI first within each experiment
res_df["EUI_sort"] = res_df["EUI"].fillna(-1.0)
res_df = res_df.sort_values(["Experiment", "Scenario", "EUI_sort"], ascending=[True, True, False]).drop(columns=["EUI_sort"])

os.makedirs(OUT_DIR, exist_ok=True)
save_path = os.path.join(OUT_DIR, "results_summary.csv")
res_df.to_csv(save_path, index=False)
print("[BIG] Saved:", save_path)

res_df.head(30)


[BIG] df_eval: 2000 | emergencies: 320
[BIG] Loaded BCResNet variants: ['bcresnet_tau1', 'bcresnet_tau3', 'bcresnet_tau8']
[BIG] Stage2 models: ['mn04_binary', 'bcresnet_tau1', 'bcresnet_tau3', 'bcresnet_tau8']
[BIG] OK Stage3_single_clips | EF1: 0.9859154929572433 | Cavg: 11607404.8
[BIG] OK S2+S3 clips: mn04_binary | EF1: 0.9905956112847633 | S3 calls: 331.0 | Cavg: 15007156.6944
[BIG] OK S2+S3 clips: bcresnet_tau1 | EF1: 0.9579288025884942 | S3 calls: 820.0 | Cavg: 8363443.168
[BIG] OK S2+S3 clips: bcresnet_tau3 | EF1: 0.891566265059743 | S3 calls: 550.0 | Cavg: 6796443.52
[BIG] OK S2+S3 clips: bcresnet_tau8 | EF1: 0.9648562300314459 | S3 calls: 1018.0 | Cavg: 9512576.2432
[BIG] OK S3 soundscape: quiet_home_day | EF1: 0.5147058823524405 | S3 rate: 1.0
[BIG] OK S3 soundscape: sleep_night | EF1: 0.499999999999475 | S3 rate: 1.0
[BIG] OK S3 soundscape: busy_indoor | EF1: 0.36496350364913743 | S3 rate: 1.0
[BIG] OK S3 soundscape: kitchen_like | EF1: 0.3235294117642811 | S3 rate: 1.0
[BI

,Experiment,Scenario,Stage1,stage2_name,pipeline,EF1,Prec,Rec,AUROC,TR,DER,TTD_sec,Cavg_MACs_per_s,EUI,Stage1_calls,Stage2_calls,Stage3_calls,Top1_Acc
55,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_A_energy,bcresnet_tau3,S1+S2+S3,0.153846,1.000000,0.083333,0.551389,2.2,0.004167,41.438344,5.485584e+06,2.804554e-08,11991.0,142.0,5.0,NaN
54,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_A_energy,bcresnet_tau1,S1+S2+S3,0.181818,1.000000,0.100000,0.550965,2.2,0.014167,41.438344,6.646325e+06,2.735620e-08,11991.0,142.0,17.0,NaN
58,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_B_energy_flux,bcresnet_tau1,S1+S2+S3,0.181818,1.000000,0.100000,0.553319,2.8,0.014167,41.438344,7.787720e+06,2.334678e-08,11991.0,180.0,17.0,NaN
59,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_B_energy_flux,bcresnet_tau3,S1+S2+S3,0.153846,1.000000,0.083333,0.556579,2.8,0.004167,41.438344,6.626980e+06,2.321512e-08,11991.0,180.0,5.0,NaN
56,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_A_energy,bcresnet_tau8,S1+S2+S3,0.171429,0.600000,0.100000,0.551637,2.2,0.040000,41.438344,9.644904e+06,1.777400e-08,11991.0,142.0,48.0,NaN
60,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_B_energy_flux,bcresnet_tau8,S1+S2+S3,0.197183,0.636364,0.116667,0.556418,2.8,0.056667,41.438344,1.272087e+07,1.550076e-08,11991.0,180.0,68.0,NaN
63,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_C_sleep_aware,bcresnet_tau3,S1+S2+S3,0.324324,0.857143,0.200000,0.697376,12.3,0.057500,1.438344,2.423155e+07,1.338438e-08,11991.0,560.0,69.0,NaN
62,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_C_sleep_aware,bcresnet_tau1,S1+S2+S3,0.320988,0.619048,0.216667,0.699817,12.3,0.055833,1.438344,2.403809e+07,1.335329e-08,11991.0,560.0,67.0,NaN
53,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_A_energy,mn04_binary,S1+S2+S3,0.179104,0.857143,0.100000,0.553874,2.2,0.005833,41.438344,1.689908e+07,1.059847e-08,11991.0,142.0,7.0,NaN
57,Stage1+Stage2+Stage3_soundscapes,busy_indoor,S1_B_energy_flux,mn04_binary,S1+S2+S3,0.179104,0.857143,0.100000,0.559561,2.8,0.005833,41.438344,2.104302e+07,8.511348e-09,11991.0,180.0,7.0,NaN


In [ ]:
# =========================
# ZIP + DOWNLOAD OUT_DIR + COPY TO GOOGLE DRIVE
# =========================
import os, subprocess
from datetime import datetime

assert "OUT_DIR" in globals(), "OUT_DIR is not defined"
out_dir = OUT_DIR
assert os.path.isdir(out_dir), f"OUT_DIR not found: {out_dir}"

# 1) Make a zip in /content
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_path = f"/content/{os.path.basename(out_dir.rstrip('/'))}_{ts}.zip"

# Use system zip (fast + reliable in Colab)
subprocess.run(["bash", "-lc", f"cd '{out_dir}' && zip -r '{zip_path}' . -q"], check=True)
print("[OK] Zipped:", zip_path)

# 2) Download to your local machine
try:
    from google.colab import files
    files.download(zip_path)
    print("[OK] Download triggered.")
except Exception as e:
    print("[WARN] files.download failed (not in Colab UI?). Error:", e)

# 3) Copy BOTH the folder and zip to Google Drive
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)

    drive_root = "/content/drive/MyDrive/echo_guard_runs_exports"
    os.makedirs(drive_root, exist_ok=True)

    dst_dir = os.path.join(drive_root, f"{os.path.basename(out_dir.rstrip('/'))}_{ts}")
    os.makedirs(dst_dir, exist_ok=True)

    # Copy folder contents (rsync keeps structure + is fast)
    subprocess.run(["bash", "-lc", f"rsync -a '{out_dir.rstrip('/')}/' '{dst_dir}/'"], check=True)

    # Copy zip too (so you have both)
    dst_zip = os.path.join(drive_root, os.path.basename(zip_path))
    subprocess.run(["bash", "-lc", f"cp '{zip_path}' '{dst_zip}'"], check=True)

    print("[OK] Copied OUT_DIR to Drive:", dst_dir)
    print("[OK] Copied ZIP to Drive:", dst_zip)

except Exception as e:
    print("[WARN] Drive copy failed. Error:", e)


[OK] Zipped: /content/echo_guard_runs_20260114_134030.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[OK] Download triggered.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[OK] Copied OUT_DIR to Drive: /content/drive/MyDrive/echo_guard_runs_exports/echo_guard_runs_20260114_134030
[OK] Copied ZIP to Drive: /content/drive/MyDrive/echo_guard_runs_exports/echo_guard_runs_20260114_134030.zip
